***Can only retrieve up to 1000 data points from Flipside API***

In [1]:
import pandas as pd
import numpy as np 
import requests
import json
import time
from flipside import Flipside
import os
import traceback
from dotenv import load_dotenv
import datetime as dt
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
#from prophet import Prophet
from dash import Dash, html, dcc, Input, Output, State, callback
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from dash import dash_table



from sql.sql_scripts import mints, sales, eth_price

In [2]:
load_dotenv()

True

In [3]:
pd.options.display.float_format = '{:,.2f}'.format


In [4]:
opensea_api_key = os.getenv('opensea_api_key')
print(opensea_api_key)

c0c9ef9df89042059cee12e5ca9197e9


***Listing Data***

In [5]:
def fetch_listings(api_key, delay_between_requests=1):
    base_url = "https://api.opensea.io/api/v2/listings/collection/3dns-powered-domains/all"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {"limit": 100} 

    listings = []
    page_count = 0

    while True:
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_listings = data.get("listings", [])
            listings.extend(fetched_listings)
            page_count += 1
            
            # Extract and print the cursor
            next_cursor = data.get("next")
            print(f"Page {page_count}, Cursor: {next_cursor}, Listings Fetched: {len(fetched_listings)}")
            
            if next_cursor:
                params['next'] = next_cursor  # Update the 'next' parameter for the next request
            else:
                break  # No more pages to fetch
                
            # Implementing delay
            time.sleep(delay_between_requests)
            
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

    print(f"Total pages fetched: {page_count}")
    print(f"Total listings fetched: {len(listings)}")
    
    df = pd.DataFrame(listings)
    return df

***Descriptions***

In [6]:
def save_last_identifier(identifier):
    with open("last_identifier.txt", "w") as file:
        file.write(identifier)

def load_last_identifier():
    try:
        with open("last_identifier.txt", "r") as file:
            return file.read().strip()
    except FileNotFoundError:
        return None

In [7]:
def fetch_all_descriptions(api_key, delay_between_requests=1):
    base_url = "https://api.opensea.io/api/v2/collection/3dns-powered-domains/nfts"
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {"limit": 100}

    all_descriptions = []

    page_count = 0
    last_identifier = load_last_identifier()

    while True:
        if last_identifier:
            params['last_identifier'] = last_identifier

        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_descriptions = data.get("nfts", [])
            
            if not fetched_descriptions:
                break

            # Process only name and identifier for each description
            for description in fetched_descriptions:
                processed_description = {
                    "name": description.get('name'),
                    "identifier": description.get('identifier')
                }
                all_descriptions.append(processed_description)
            
            # Update the last_identifier to the latest one fetched
            last_identifier = fetched_descriptions[-1].get('identifier')
            save_last_identifier(last_identifier)
            
            page_count += 1
            next_cursor = data.get("next")
            print(f"Page {page_count}, Cursor: {next_cursor} Descriptions Fetched: {len(fetched_descriptions)}, total fetched: {len(all_descriptions)}")
            
            if next_cursor:
                params['next'] = next_cursor
            else:
                break  # No more pages to fetch

            time.sleep(delay_between_requests)
        else:
            print(f"Failed to fetch data: {response.status_code}")
            break

    print(f"Total pages fetched: {page_count}, Total descriptions fetched: {len(all_descriptions)}")
    
    # Save the processed descriptions to a file
    df = pd.DataFrame(all_descriptions)
    return df

***Events***

In [8]:
import json
import os

def save_last_timestamp(event_type, timestamp):
    data = {}
    if os.path.exists("last_timestamps.json"):
        with open("last_timestamps.json", "r") as file:
            data = json.load(file)
    data[event_type] = timestamp
    with open("last_timestamps.json", "w") as file:
        json.dump(data, file)

def load_last_timestamp(event_type):
    if os.path.exists("last_timestamps.json"):
        with open("last_timestamps.json", "r") as file:
            data = json.load(file)
        return data.get(event_type, None)
    return None

In [9]:
def fetch_event_type(api_key, event_type, all_events, params, headers):
    base_url = f"https://api.opensea.io/api/v2/events/collection/3dns-powered-domains"
    params['event_type'] = event_type
    
    # Load the last timestamp/identifier
    last_timestamp = load_last_timestamp(event_type)
    if last_timestamp:
        params['occurred_after'] = last_timestamp
    
    page_count = 0
    while True:
        response = requests.get(base_url, headers=headers, params=params)
        if response.status_code == 200:
            data = response.json()
            fetched_events = data.get("asset_events", [])
            all_events.extend(fetched_events)
            
            if fetched_events:
                # Update the last timestamp/identifier to the latest one fetched
                last_event_time = fetched_events[-1].get("created_date")
                save_last_timestamp(event_type, last_event_time)
            
            page_count += 1
            next_cursor = data.get("next")
            print(f"Fetching {event_type}: Page {page_count}, Events Fetched: {len(fetched_events)}, Total Events: {len(all_events)}, next cursor: {next_cursor}")
            
            if next_cursor:
                params['next'] = next_cursor
            else:
                break  # No more pages to fetch

            time.sleep(1)  # Delay between requests
        else:
            print(f"Failed to fetch {event_type} data: HTTP {response.status_code}, Response: {response.text}")
            break

def fetch_all_events(api_key):
    headers = {
        "accept": "application/json",
        "x-api-key": api_key
    }
    params = {
        "limit": 50  # Adjust the limit as needed
    }

    all_events = []

    # Fetch listings
    fetch_event_type(api_key, "listing", all_events, params.copy(), headers)

    # Fetch sales
    fetch_event_type(api_key, "sale", all_events, params.copy(), headers)

    # Save the fetched events to a DataFrame
    print(f"Total events fetched: {len(all_events)}")
    df = pd.DataFrame(all_events)
    return df 

***Flipside Data***

In [10]:
flipside_api_key = os.getenv("FLIPSIDE_API_KEY")
flipside = Flipside(flipside_api_key, "https://api-v2.flipsidecrypto.xyz")

In [11]:
def flipside_api_results(query):
  query_result_set = flipside.query(query)
  # what page are we starting on?
  current_page_number = 1

  # How many records do we want to return in the page?
  page_size = 1000

  # set total pages to 1 higher than the `current_page_number` until
  # we receive the total pages from `get_query_results` given the 
  # provided `page_size` (total_pages is dynamically determined by the API 
  # based on the `page_size` you provide)

  total_pages = 2


  # we'll store all the page results in `all_rows`
  all_rows = []

  while current_page_number <= total_pages:
    results = flipside.get_query_results(
      query_result_set.query_id,
      page_number=current_page_number,
      page_size=page_size
    )

    total_pages = results.page.totalPages
    if results.records:
        all_rows = all_rows + results.records
    
    current_page_number += 1

  return pd.DataFrame(all_rows)

query_result_set = flipside.query(sales)

# what page are we starting on?
current_page_number = 1

# How many records do we want to return in the page?
page_size = 1000

# set total pages to 1 higher than the `current_page_number` until
# we receive the total pages from `get_query_results` given the 
# provided `page_size` (total_pages is dynamically determined by the API 
# based on the `page_size` you provide)
total_pages = 2

# we'll store all the page results in `all_rows`
all_rows = []

while current_page_number <= total_pages:
  results = flipside.get_query_results(
    query_result_set.query_id,
    page_number=current_page_number,
    page_size=page_size
  )

  total_pages = results.page.totalPages
  if results.records:
      all_rows = all_rows + results.records
  
  current_page_number += 1

mints_df = pd.DataFrame(all_rows)



def createQueryRun(sql):
    url = "https://api-v2.flipsidecrypto.xyz/json-rpc"
    payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "createQueryRun",
        "params": [{
            "resultTTLHours": 1,
            "maxAgeMinutes": 0,
            "sql": sql,
            "tags": {"source": "streamlit-demo", "env": "test"},
            "dataSource": "snowflake-default",
            "dataProvider": "flipside"
        }],
        "id": 1
    })
    headers = {'Content-Type': 'application/json', 'x-api-key': flipside_api_key}
    response = requests.post(url, headers=headers, data=payload)
    response_data = response.json()

    # Check for errors in the response
    if 'error' in response_data:
        error_message = response_data['error'].get('message', 'No error message provided')
        print("Full response data for debugging:", response_data)
        raise Exception(f"Error: {error_message}")

    if 'result' not in response_data or 'queryRun' not in response_data['result'] or 'id' not in response_data['result']['queryRun']:
        print("Unexpected response structure:", response_data)
        raise Exception("Unexpected response structure")

    query_run_id = response_data['result']['queryRun']['id']
    return response_data, query_run_id

#@st.cache_data(ttl='15m')
def getQueryResults(query_run_id, attempts=10, delay=30):
    url = "https://api-v2.flipsidecrypto.xyz/json-rpc"
    payload = json.dumps({
        "jsonrpc": "2.0",
        "method": "getQueryRunResults",
        "params": [{"queryRunId": query_run_id, "format": "json", "page": {"number": 1, "size": 10000}}],
        "id": 1
    })
    headers = {'Content-Type': 'application/json', 'x-api-key': flipside_api_key}

    for attempt in range(attempts):
        response = requests.post(url, headers=headers, data=payload)
        resp_json = response.json()
        if 'result' in resp_json:
            return resp_json  # Data is ready
        elif 'error' in resp_json and 'message' in resp_json['error'] and 'not yet completed' in resp_json['error']['message']:
            time.sleep(delay)  # Wait for a bit before retrying
        else:
            print("Unexpected response or error:", resp_json)
            break  # Break on unexpected error
    return None  # Return None if data isn't ready after all attempts


def retrieve_flipside_data(query):
    try:
        response_data, q_id = createQueryRun(query)
        if q_id:
            df_json = getQueryResults(q_id)
            if df_json:
                df = pd.DataFrame(df_json['result']['rows'])
                print(f"data fetched: {df.head()}")
                return df
            else:
                print('Failed to get results')
        else:
            print('Failed to create query run')
    except Exception as e:
        print(f"Error in fetching data: {e}")
        traceback.print_exc()
    

***Data Retrieval/Processing***

In [12]:
mint_df = flipside_api_results(mints)

In [13]:
mint_df

,day,tokenid,tx_hash,__row_index
0,2024-07-10T10:00:00.000Z,"8,350,705,726,613,897,402,252,190,637,230,342,5...",0x49c925349318bcbf089b910c7cbb9b75dcff8fd1b992...,0
1,2024-07-10T10:00:00.000Z,"8,350,705,726,613,897,402,252,190,637,230,342,5...",0x49c925349318bcbf089b910c7cbb9b75dcff8fd1b992...,1
2,2024-07-10T09:00:00.000Z,"831,581,688,049,173,449,027,930,491,313,066,372...",0x797479e07d791a0f79d0433a514dbb4fcd196e57a047...,2
3,2024-07-10T09:00:00.000Z,"831,581,688,049,173,449,027,930,491,313,066,372...",0x797479e07d791a0f79d0433a514dbb4fcd196e57a047...,3
4,2024-07-10T04:00:00.000Z,"50,886,887,701,499,123,067,873,961,026,079,613,...",0x4b19b2871109c40c80ae709a599edea386639794904e...,4
...,...,...,...,...
32817,2023-10-04T02:00:00.000Z,"73,079,797,870,479,156,483,046,577,349,357,812,...",0x981dd9c93e0419a227128b7ee95c620e7dbc88fe92e8...,32817
32818,2023-10-04T02:00:00.000Z,"97,049,678,982,547,095,836,606,605,282,336,114,...",0xd7ece008b97cad61f02396def394751c3fa9a4c2844a...,32818
32819,2023-10-04T02:00:00.000Z,"97,049,678,982,547,095,836,606,605,282,336,114,...",0xd7ece008b97cad61f02396def394751c3fa9a4c2844a...,32819
32820,2023-10-04T02:00:00.000Z,"77,875,674,875,312,083,922,572,966,307,662,148,...",0x64f70f46174e15001ff109ce325f9d0e0d7a571a22ed...,32820


In [14]:
sales_df = flipside_api_results(sales)
# sales_df.to_csv('data/sales_data.csv')


sales_df = pd.read_csv('data/sales_data.csv')
sales_df

In [15]:
eth_usd_df = flipside_api_results(eth_price)

In [16]:
events_df = fetch_all_events(api_key= opensea_api_key)
#events_df.to_csv('data/events_data.csv')

Fetching listing: Page 1, Events Fetched: 50, Total Events: 50, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA3LTA3KzA5JTNBMjglM0ExNy42MzM2NzkmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjQxMjExMTIzODA=
Fetching listing: Page 2, Events Fetched: 50, Total Events: 100, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA3LTAxKzE1JTNBMTElM0EzOC42MzA4MDkmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM5NDU0NzQ0NDA=
Fetching listing: Page 3, Events Fetched: 50, Total Events: 150, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTI3KzAyJTNBMDklM0EyMC4wOTg4NTImLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM4MTcxOTM5MzQ=
Fetching listing: Page 4, Events Fetched: 50, Total Events: 200, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTIyKzE1JTNBMTYlM0EyNi42Mzk1NTQmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM2ODkzNTA3MzU=
Fetching listing: Page 5, Events Fetched: 50, Total Events: 250, next cursor: LWV2ZW50X3RpbWVzdGFtcD0yMDI0LTA2LTIyKzE0JTNBMDglM0EwNi44ODgxMTkmLWV2ZW50X3R5cGU9Y3JlYXRlZCYtcGs9MjM2ODgwMTU5NjQ=
Fetching listing: Page 6, Events Fetched: 50, 

events_df = pd.read_csv('data/events_data.csv')
events_df

In [17]:
descriptions_df = fetch_all_descriptions(api_key= opensea_api_key)
#descriptions_df.to_csv('data/descriptions_data.csv')

Page 1, Cursor: LXBrPTE5MDE2NTczMzk= Descriptions Fetched: 100, total fetched: 100
Page 2, Cursor: LXBrPTE4OTYyNzU4NzU= Descriptions Fetched: 100, total fetched: 200
Page 3, Cursor: LXBrPTE4ODQ5NzI0MTQ= Descriptions Fetched: 100, total fetched: 300
Page 4, Cursor: LXBrPTE4NzYzMDk2NjU= Descriptions Fetched: 100, total fetched: 400
Page 5, Cursor: LXBrPTE4Njc3MDQ3OTQ= Descriptions Fetched: 100, total fetched: 500
Page 6, Cursor: LXBrPTE4NTk2MDk5MjY= Descriptions Fetched: 100, total fetched: 600
Page 7, Cursor: LXBrPTE4NDgzNjQ0NTU= Descriptions Fetched: 100, total fetched: 700
Page 8, Cursor: LXBrPTE4Mzg0MTI2MTY= Descriptions Fetched: 100, total fetched: 800
Page 9, Cursor: LXBrPTE4MzYxNjM3MzA= Descriptions Fetched: 100, total fetched: 900
Page 10, Cursor: LXBrPTE4MzM0OTc1NDQ= Descriptions Fetched: 100, total fetched: 1000
Page 11, Cursor: LXBrPTE4MzAxNzM2NDE= Descriptions Fetched: 100, total fetched: 1100
Page 12, Cursor: LXBrPTE4MjcyNzEzMjk= Descriptions Fetched: 100, total fetched: 120

In [21]:
#descriptions_df = pd.read_csv('data/descriptions_data.csv')
descriptions_df['name'].tail(20)

16390     tokentroopers.xyz
16391            mudbox.xyz
16392     beallclothing.com
16393          brandley.xyz
16394        tameimpala.xyz
16395          efferium.org
16396           wizkika.com
16397    onchaindomains.org
16398              3dns.box
16399                   box
16400               finance
16401                   inc
16402                  link
16403                   wtf
16404                   xyz
16405                  tech
16406                    io
16407                   org
16408                   net
16409                   com
Name: name, dtype: object

In [22]:
listings_df = fetch_listings(api_key= opensea_api_key, delay_between_requests=1)


Page 1, Cursor: cGs9MTc0ODI3MTAyMDYmY3JlYXRlZF9kYXRlPTIwMjQtMDMtMDYrMDMlM0E1MiUzQTMyLjU4ODA4Nw==, Listings Fetched: 100
Page 2, Cursor: cGs9MTg2NDE4NDExNDMmY3JlYXRlZF9kYXRlPTIwMjQtMDQtMjgrMTElM0EyNiUzQTM1LjEwMTg2MQ==, Listings Fetched: 100
Page 3, Cursor: cGs9MTg3OTI1MTI2MjYmY3JlYXRlZF9kYXRlPTIwMjQtMDUtMDQrMTUlM0E1NyUzQTA4LjYyNjYxOA==, Listings Fetched: 100
Page 4, Cursor: cGs9MTkzNDAzNTMxOTMmY3JlYXRlZF9kYXRlPTIwMjQtMDUtMjcrMTMlM0E0NSUzQTMxLjM5OTk0Mw==, Listings Fetched: 100
Page 5, Cursor: cGs9MTk3NzY3MjQ4NzImY3JlYXRlZF9kYXRlPTIwMjQtMDYtMTUrMTMlM0E1NyUzQTIzLjc1MTUyNw==, Listings Fetched: 100
Page 6, Cursor: cGs9MTk5NzQ2ODQyOTYmY3JlYXRlZF9kYXRlPTIwMjQtMDYtMjQrMDglM0E0NSUzQTIwLjc2NTIwMg==, Listings Fetched: 100
Page 7, Cursor: cGs9MjAzMjYwNzk4MzAmY3JlYXRlZF9kYXRlPTIwMjQtMDctMDgrMjAlM0ExMCUzQTUwLjk2MTI2OQ==, Listings Fetched: 100
Page 8, Cursor: None, Listings Fetched: 9
Total pages fetched: 8
Total listings fetched: 709


listings_df = pd.read_csv('data/listings_data.csv')
listings_df

In [23]:
def unpack_protocol_data(row):
    protocol_data = row['protocol_data']
    parameters = protocol_data.get('parameters', {})
    consideration = parameters.get('consideration', [{}])
    offer = parameters.get('offer', [{}])
    price = row['price']['current']
    
    chain = row['chain']
    order_hash = row['order_hash']
    currency = price.get('currency')
    price_string = price.get('value')
    price_in_eth = float(price_string) / (10 ** price.get('decimals', 18))
    primary_recipient = consideration[0].get('recipient') if consideration else None
    identifier_or_criteria = offer[0].get('identifierOrCriteria') if offer else None
    start_time = parameters.get('startTime')
    end_time = parameters.get('endTime')
    
    return pd.Series([
        chain, order_hash, currency, price_string, price_in_eth, 
        primary_recipient, identifier_or_criteria, start_time, end_time
    ])

In [24]:
listings_df.columns

Index(['order_hash', 'chain', 'type', 'price', 'protocol_data',
       'protocol_address'],
      dtype='object')

In [25]:
unpacked_columns = listings_df.apply(unpack_protocol_data, axis=1)
unpacked_columns.columns = [
    'chain', 'order_hash', 'currency', 'price_string', 'price_in_eth', 
    'primary_recipient', 'identifier_or_criteria', 'start_time', 'end_time'
]
listings_df = pd.concat([listings_df, unpacked_columns], axis=1)
listings_df = listings_df.drop(columns=['protocol_data'])


In [26]:
listings_df.to_csv('data/listings_data.csv')

In [27]:
listings_df = pd.read_csv('data/listings_data.csv')
listings_df

,Unnamed: 0,order_hash,chain,type,price,protocol_address,chain.1,order_hash.1,currency,price_string,price_in_eth,primary_recipient,identifier_or_criteria,start_time,end_time
0,0,0x902ff72f57b579d6a2fac92a0714ca5274172ccdeb51...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",0x00000000000000adc04c56bf30ac9d3c0aaf14dc,optimism,0x902ff72f57b579d6a2fac92a0714ca5274172ccdeb51...,ETH,5000000000000000000,5.00,0x64233eAa064ef0d54ff1A963933D0D2d46ab5829,2497682056220887489943867884633909011520297462...,1705460459,1721181652
1,1,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",0x00000000000000adc04c56bf30ac9d3c0aaf14dc,optimism,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,ETH,8000000000000000000,8.00,0x47baba9b83c7Cd484297e771d99076e904647511,1033482404727849386934500727220714048602002576...,1705973310,1721694506
2,2,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",0x00000000000000adc04c56bf30ac9d3c0aaf14dc,optimism,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,ETH,3000000000000000000,3.00,0x97F6F6C5503b9Fe33cD774281be3e92ED1ee3b6f,1022958450261355013510435244537320586681820335...,1706042993,1721767789
3,3,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",0x00000000000000adc04c56bf30ac9d3c0aaf14dc,optimism,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,ETH,3000000000000000000,3.00,0x97F6F6C5503b9Fe33cD774281be3e92ED1ee3b6f,7527017910112430367015527233527204966333609402...,1706043120,1721767916
4,4,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",0x00000000000000adc04c56bf30ac9d3c0aaf14dc,optimism,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,ETH,8000000000000000000,8.00,0x99D104C7A9D51953bfa13a3e85feDad2E8b858C2,1535191139414816108344131948174266971965245087...,1706069852,1721791047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
704,704,0xbf33c4bcd70b09ffc2d32d45f3f683b0f006df5e617c...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",0x0000000000000068f116a894984e2db1123eb395,optimism,0xbf33c4bcd70b09ffc2d32d45f3f683b0f006df5e617c...,ETH,1000000000000000000,1.00,0x8699B8b6C1163f29F6ab153BF890c0214D2d4df4,4442411053805375666455962090072480703921829609...,1720527863,1723206142
705,705,0xd76ea0933059e5d358e6768a87ac0358d4f7b6b3b77e...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",0x0000000000000068f116a894984e2db1123eb395,optimism,0xd76ea0933059e5d358e6768a87ac0358d4f7b6b3b77e...,ETH,960000000000000000,0.96,0x8699B8b6C1163f29F6ab153BF890c0214D2d4df4,1114682888164222083435423137837325017955370624...,1720527863,1723206142
706,706,0x3682a15b341d33dd93f859a3029596e346769720a6ea...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",0x0000000000000068f116a894984e2db1123eb395,optimism,0x3682a15b341d33dd93f859a3029596e346769720a6ea...,ETH,1690000000000000000,1.69,0x8699B8b6C1163f29F6ab153BF890c0214D2d4df4,2777096508780913048630830342778877876153672428...,1720527777,1720786974
707,707,0xaedefdd5ee2c58433e4135132b5b8454653725d40859...,optimism,basic,"{'current': {'currency': 'ETH', 'decimals': 18...",0x0000000000000068f116a894984e2db1123eb395,optimism,0xaedefdd5ee2c58433e4135132b5b8454653725d40859...,ETH,1000000000000000000,1.00,0x9a93aE4F408a1e7D178650ca9f7Ab2135c512EDA,2836046696501367752144150720332275946415977732...,1720561057,1721165912


In [28]:
listings_df['identifier_or_criteria'] = listings_df['identifier_or_criteria'].astype(float)
listings_df.rename(columns={'identifier_or_criteria':'tokenid'}, inplace=True)

In [29]:
listings_df.drop(columns=['price_string','currency','primary_recipient','chain','price','protocol_address','type'], inplace=True)

In [30]:
descriptions_df.rename(columns={'identifier':'tokenid'}, inplace=True)

In [31]:
descriptions_df['tokenid'] = descriptions_df['tokenid'].astype(float)

In [32]:
listings_with_names = listings_df.merge(descriptions_df, how='left', on='tokenid')
listings_with_names

,Unnamed: 0,order_hash,chain.1,order_hash.1,price_in_eth,tokenid,start_time,end_time,name
0,0,0x902ff72f57b579d6a2fac92a0714ca5274172ccdeb51...,optimism,0x902ff72f57b579d6a2fac92a0714ca5274172ccdeb51...,5.00,"24,976,820,562,208,875,018,004,955,422,466,815,...",1705460459,1721181652,eth.attorney
1,1,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,optimism,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,8.00,"103,348,240,472,784,936,049,086,084,648,101,639...",1705973310,1721694506,open.box
2,2,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,optimism,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,3.00,"102,295,845,026,135,496,883,892,522,779,159,999...",1706042993,1721767789,jewelry.box
3,3,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,optimism,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,3.00,"75,270,179,101,124,303,647,088,469,843,731,665,...",1706043120,1721767916,gay.box
4,4,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,optimism,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,8.00,"15,351,911,394,148,160,117,736,899,619,150,406,...",1706069852,1721791047,power.box
...,...,...,...,...,...,...,...,...,...
704,704,0xbf33c4bcd70b09ffc2d32d45f3f683b0f006df5e617c...,optimism,0xbf33c4bcd70b09ffc2d32d45f3f683b0f006df5e617c...,1.00,"444,241,105,380,537,591,620,801,961,800,665,220...",1720527863,1723206142,demon.box
705,705,0xd76ea0933059e5d358e6768a87ac0358d4f7b6b3b77e...,optimism,0xd76ea0933059e5d358e6768a87ac0358d4f7b6b3b77e...,0.96,"111,468,288,816,422,214,204,408,193,882,129,338...",1720527863,1723206142,girl.box
706,706,0x3682a15b341d33dd93f859a3029596e346769720a6ea...,optimism,0x3682a15b341d33dd93f859a3029596e346769720a6ea...,1.69,"27,770,965,087,809,130,203,304,471,643,289,691,...",1720527777,1720786974,ia.box
707,707,0xaedefdd5ee2c58433e4135132b5b8454653725d40859...,optimism,0xaedefdd5ee2c58433e4135132b5b8454653725d40859...,1.00,"28,360,466,965,013,677,414,172,494,359,945,146,...",1720561057,1721165912,cypherpunk.box


In [33]:
box_listings = listings_with_names[listings_with_names['name'].str.endswith('.box')]
box_listings

,Unnamed: 0,order_hash,chain.1,order_hash.1,price_in_eth,tokenid,start_time,end_time,name
1,1,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,optimism,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,8.00,"103,348,240,472,784,936,049,086,084,648,101,639...",1705973310,1721694506,open.box
2,2,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,optimism,0x3040ad3f37193e684cb2baa0811d21cd345108df5a2a...,3.00,"102,295,845,026,135,496,883,892,522,779,159,999...",1706042993,1721767789,jewelry.box
3,3,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,optimism,0xb62518c46df32c07046a53f3df61112cc4460bc648ac...,3.00,"75,270,179,101,124,303,647,088,469,843,731,665,...",1706043120,1721767916,gay.box
4,4,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,optimism,0xe4c2e069a35f4b943e449d8e510f952ff3cb2fc133d3...,8.00,"15,351,911,394,148,160,117,736,899,619,150,406,...",1706069852,1721791047,power.box
5,5,0x8bc67ab0f6355d8c78a317dee43dd93aad665ffa66d0...,optimism,0x8bc67ab0f6355d8c78a317dee43dd93aad665ffa66d0...,69.00,"92,045,738,473,223,622,079,151,157,139,670,821,...",1706083009,1721804202,dan.box
...,...,...,...,...,...,...,...,...,...
704,704,0xbf33c4bcd70b09ffc2d32d45f3f683b0f006df5e617c...,optimism,0xbf33c4bcd70b09ffc2d32d45f3f683b0f006df5e617c...,1.00,"444,241,105,380,537,591,620,801,961,800,665,220...",1720527863,1723206142,demon.box
705,705,0xd76ea0933059e5d358e6768a87ac0358d4f7b6b3b77e...,optimism,0xd76ea0933059e5d358e6768a87ac0358d4f7b6b3b77e...,0.96,"111,468,288,816,422,214,204,408,193,882,129,338...",1720527863,1723206142,girl.box
706,706,0x3682a15b341d33dd93f859a3029596e346769720a6ea...,optimism,0x3682a15b341d33dd93f859a3029596e346769720a6ea...,1.69,"27,770,965,087,809,130,203,304,471,643,289,691,...",1720527777,1720786974,ia.box
707,707,0xaedefdd5ee2c58433e4135132b5b8454653725d40859...,optimism,0xaedefdd5ee2c58433e4135132b5b8454653725d40859...,1.00,"28,360,466,965,013,677,414,172,494,359,945,146,...",1720561057,1721165912,cypherpunk.box


In [34]:
events_df_copy = events_df.copy()
events_df_copy

,event_type,order_hash,order_type,chain,protocol_address,start_date,expiration_date,asset,quantity,maker,taker,payment,criteria,event_timestamp,is_private_listing,closing_date,nft,seller,buyer,transaction
0,order,0x49729029191017c544a3f67f03968dd12bf16c69e1dd...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,562,975.00","1,736,460,564.00",{'identifier': '106968891900401063741132344938...,1,0x386ae4d6db89e0bbd41ef8cba13460edcf867420,,"{'quantity': '33000000000000000000', 'token_ad...",{},1720563049,False,NaN,NaN,NaN,NaN,NaN
1,order,0xaedefdd5ee2c58433e4135132b5b8454653725d40859...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,561,057.00","1,721,165,912.00",{'identifier': '283604669650136775214415072033...,1,0x9a93ae4f408a1e7d178650ca9f7ab2135c512eda,,"{'quantity': '1000000000000000000', 'token_add...",{},1720561062,False,NaN,NaN,NaN,NaN,NaN
2,order,0x9d15ad778c1865fa87e96f6fb85f90bed32d784c1162...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,544,932.00","1,720,612,765.00",{'identifier': '112995818248598227413563035922...,1,0x9a93ae4f408a1e7d178650ca9f7ab2135c512eda,,"{'quantity': '77700000000000000', 'token_addre...",{},1720544938,False,NaN,NaN,NaN,NaN,NaN
3,order,0x3682a15b341d33dd93f859a3029596e346769720a6ea...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,527,777.00","1,720,786,974.00",{'identifier': '277709650878091304863083034277...,1,0x8699b8b6c1163f29f6ab153bf890c0214d2d4df4,,"{'quantity': '1690000000000000000', 'token_add...",{},1720527923,False,NaN,NaN,NaN,NaN,NaN
4,order,0xd76ea0933059e5d358e6768a87ac0358d4f7b6b3b77e...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,527,863.00","1,723,206,142.00",{'identifier': '111468288816422208343542313783...,1,0x8699b8b6c1163f29f6ab153bf890c0214d2d4df4,,"{'quantity': '960000000000000000', 'token_addr...",{},1720527874,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2993,sale,0x8bde44f2d46537cf8d025cc498eaf7bdeadce3631f15...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '1000000000000000', 'token_addres...",NaN,1706970257,NaN,"1,706,970,257.00",{'identifier': '260923785748676206958654735781...,0xcc0e0b9ebd4ce352f5a50b3ae77ea01202c284ee,0xc48a8ac51df7b71e500a5580b89e47ab26ca1472,0x60e952a8e0fa8d052791b2e4a22fd52eb3c759511e62...
2994,sale,0xc0547b5b8d575e3ba16b529ee2530a35c437bc75fa93...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '110000000000000000', 'token_addr...",NaN,1706653465,NaN,"1,706,653,465.00",{'identifier': '566479911085778512201556021651...,0x240ad467a71210629d71d4de22ebde27951c83fc,0x82eb45562f991329ed2867f43fc60f0ba52c3dab,0x6672189deb7829838aabba20ddeb66a3cb5fee5b55a2...
2995,sale,0xff4b9bdd3f140f8211864ef34e0a07ea0a92eaf98f7d...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '10000000000000000', 'token_addre...",NaN,1706203283,NaN,"1,706,203,283.00",{'identifier': '364706086468987072255605061503...,0x7b363822d744143d51c69b0757165c987a6e4ef4,0xe68e8cc7ff772b026c062b9cc28246676d044947,0xd8e71f4d52c43fa770b9aa7ad7d5cfb073f04c7495e0...
2996,sale,0x03188e15aaf9596307117e30355f6b2ac79665fbd1c2...,NaN,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,NaN,NaN,NaN,1,NaN,NaN,"{'quantity': '150000000000000000', 'token_addr...",NaN,1704080589,NaN,"1,704,080,589.00",{'identifier': '992340878683635895851755157396...,0x31b32020fb1bdf0228c58b80590f07b235f2b0ce,0x64233eaa064ef0d54ff1a963933d0d2d46ab5829,0x186d1ceacf620bab710d11cf928b79e5dd46790ff1d5...


In [35]:
bids = events_df_copy[events_df_copy['event_type'] == 'order']
sales = events_df_copy[events_df_copy['event_type'] == 'sale']

bids

,event_type,order_hash,order_type,chain,protocol_address,start_date,expiration_date,asset,quantity,maker,taker,payment,criteria,event_timestamp,is_private_listing,closing_date,nft,seller,buyer,transaction
0,order,0x49729029191017c544a3f67f03968dd12bf16c69e1dd...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,562,975.00","1,736,460,564.00",{'identifier': '106968891900401063741132344938...,1,0x386ae4d6db89e0bbd41ef8cba13460edcf867420,,"{'quantity': '33000000000000000000', 'token_ad...",{},1720563049,False,NaN,NaN,NaN,NaN,NaN
1,order,0xaedefdd5ee2c58433e4135132b5b8454653725d40859...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,561,057.00","1,721,165,912.00",{'identifier': '283604669650136775214415072033...,1,0x9a93ae4f408a1e7d178650ca9f7ab2135c512eda,,"{'quantity': '1000000000000000000', 'token_add...",{},1720561062,False,NaN,NaN,NaN,NaN,NaN
2,order,0x9d15ad778c1865fa87e96f6fb85f90bed32d784c1162...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,544,932.00","1,720,612,765.00",{'identifier': '112995818248598227413563035922...,1,0x9a93ae4f408a1e7d178650ca9f7ab2135c512eda,,"{'quantity': '77700000000000000', 'token_addre...",{},1720544938,False,NaN,NaN,NaN,NaN,NaN
3,order,0x3682a15b341d33dd93f859a3029596e346769720a6ea...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,527,777.00","1,720,786,974.00",{'identifier': '277709650878091304863083034277...,1,0x8699b8b6c1163f29f6ab153bf890c0214d2d4df4,,"{'quantity': '1690000000000000000', 'token_add...",{},1720527923,False,NaN,NaN,NaN,NaN,NaN
4,order,0xd76ea0933059e5d358e6768a87ac0358d4f7b6b3b77e...,listing,optimism,0x0000000000000068f116a894984e2db1123eb395,"1,720,527,863.00","1,723,206,142.00",{'identifier': '111468288816422208343542313783...,1,0x8699b8b6c1163f29f6ab153bf890c0214d2d4df4,,"{'quantity': '960000000000000000', 'token_addr...",{},1720527874,False,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,order,0x3bbb21c5a1766b47ea91b2dafd73e7cb28198b41b3c6...,listing,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,"1,703,615,460.00","1,706,293,860.00",{'identifier': '596691372334868032834900148779...,1,0xec00481d13a4028e0b76b031cf5c863604cb2d4b,,"{'quantity': '3000000000000000000', 'token_add...",{},1703615466,False,NaN,NaN,NaN,NaN,NaN
2900,order,0xa2931b385c3bd5c237a44cc39a452e9f12499ba6cafd...,listing,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,"1,703,615,375.00","1,706,293,775.00",{'identifier': '178657174778441322562715723560...,1,0xec00481d13a4028e0b76b031cf5c863604cb2d4b,,"{'quantity': '3000000000000000000', 'token_add...",{},1703615382,False,NaN,NaN,NaN,NaN,NaN
2901,order,0x0fff804500518519b60164f5a10ef76e63da9ee77775...,listing,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,"1,703,615,237.00","1,706,293,637.00",{'identifier': '887409359338823895470224549460...,1,0x64233eaa064ef0d54ff1a963933d0d2d46ab5829,,"{'quantity': '3000000000000000000', 'token_add...",{},1703615249,False,NaN,NaN,NaN,NaN,NaN
2902,order,0x3e6ab1b4b98dd45a1cd97fc141659077293095161497...,listing,optimism,0x00000000000000adc04c56bf30ac9d3c0aaf14dc,"1,703,607,296.00","1,704,212,091.00",{'identifier': '117315147600270829665702926801...,1,0x7af4ca674c86ac74d15ae668ae7ffbe4f7ddafbf,,"{'quantity': '1000000000000000000', 'token_add...",{},1703607304,False,NaN,NaN,NaN,NaN,NaN


In [36]:
bids['identifier'] = bids['asset'].apply(lambda x: x.get('identifier') if isinstance(x, dict) else None)
sales['identifier'] = sales['nft'].apply(lambda x: x.get('identifier') if isinstance(x, dict) else None)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\2668839905.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bids['identifier'] = bids['asset'].apply(lambda x: x.get('identifier') if isinstance(x, dict) else None)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\2668839905.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['identifier'] = sales['nft'].apply(lambda x: x.get('identifier') if isinstance(x, dict) else None)


In [37]:
bids['identifier'] = bids['identifier'].astype(float)
bids.rename(columns={'identifier':'tokenid'}, inplace=True) 


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\46667634.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bids['identifier'] = bids['identifier'].astype(float)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\46667634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bids.rename(columns={'identifier':'tokenid'}, inplace=True)


In [38]:
bids = bids.merge(descriptions_df, how='left', on='tokenid')

In [39]:
bids.columns

Index(['event_type', 'order_hash', 'order_type', 'chain', 'protocol_address',
       'start_date', 'expiration_date', 'asset', 'quantity', 'maker', 'taker',
       'payment', 'criteria', 'event_timestamp', 'is_private_listing',
       'closing_date', 'nft', 'seller', 'buyer', 'transaction', 'tokenid',
       'name'],
      dtype='object')

In [40]:
bids.drop(columns=['protocol_address','chain','maker','criteria','is_private_listing','closing_date','nft','seller','buyer'], inplace=True)

In [41]:
sales['identifier'] = sales['identifier'].astype(float)
sales.rename(columns={'identifier':'tokenid'}, inplace=True)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\3642358893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales['identifier'] = sales['identifier'].astype(float)
C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\3642358893.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sales.rename(columns={'identifier':'tokenid'}, inplace=True)


In [42]:
sales = sales.merge(descriptions_df, how='left', on='tokenid')
sales.drop(columns=['order_type', 'chain','start_date','expiration_date','asset','maker','is_private_listing','protocol_address','criteria'], inplace=True)

In [43]:
bids

,event_type,order_hash,order_type,start_date,expiration_date,asset,quantity,taker,payment,event_timestamp,transaction,tokenid,name
0,order,0x49729029191017c544a3f67f03968dd12bf16c69e1dd...,listing,"1,720,562,975.00","1,736,460,564.00",{'identifier': '106968891900401063741132344938...,1,,"{'quantity': '33000000000000000000', 'token_ad...",1720563049,NaN,"106,968,891,900,401,057,912,681,673,593,283,028...",blum.box
1,order,0xaedefdd5ee2c58433e4135132b5b8454653725d40859...,listing,"1,720,561,057.00","1,721,165,912.00",{'identifier': '283604669650136775214415072033...,1,,"{'quantity': '1000000000000000000', 'token_add...",1720561062,NaN,"28,360,466,965,013,677,414,172,494,359,945,146,...",cypherpunk.box
2,order,0x9d15ad778c1865fa87e96f6fb85f90bed32d784c1162...,listing,"1,720,544,932.00","1,720,612,765.00",{'identifier': '112995818248598227413563035922...,1,,"{'quantity': '77700000000000000', 'token_addre...",1720544938,NaN,"112,995,818,248,598,233,060,911,981,365,146,698...",777777.box
3,order,0x3682a15b341d33dd93f859a3029596e346769720a6ea...,listing,"1,720,527,777.00","1,720,786,974.00",{'identifier': '277709650878091304863083034277...,1,,"{'quantity': '1690000000000000000', 'token_add...",1720527923,NaN,"27,770,965,087,809,130,203,304,471,643,289,691,...",ia.box
4,order,0xd76ea0933059e5d358e6768a87ac0358d4f7b6b3b77e...,listing,"1,720,527,863.00","1,723,206,142.00",{'identifier': '111468288816422208343542313783...,1,,"{'quantity': '960000000000000000', 'token_addr...",1720527874,NaN,"111,468,288,816,422,214,204,408,193,882,129,338...",girl.box
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2899,order,0x3bbb21c5a1766b47ea91b2dafd73e7cb28198b41b3c6...,listing,"1,703,615,460.00","1,706,293,860.00",{'identifier': '596691372334868032834900148779...,1,,"{'quantity': '3000000000000000000', 'token_add...",1703615466,NaN,"59,669,137,233,486,797,576,913,893,823,506,075,...",mintdomains.io
2900,order,0xa2931b385c3bd5c237a44cc39a452e9f12499ba6cafd...,listing,"1,703,615,375.00","1,706,293,775.00",{'identifier': '178657174778441322562715723560...,1,,"{'quantity': '3000000000000000000', 'token_add...",1703615382,NaN,"17,865,717,477,844,132,496,699,921,146,330,333,...",dnsloans.com
2901,order,0x0fff804500518519b60164f5a10ef76e63da9ee77775...,listing,"1,703,615,237.00","1,706,293,637.00",{'identifier': '887409359338823895470224549460...,1,,"{'quantity': '3000000000000000000', 'token_add...",1703615249,NaN,"8,874,093,593,388,239,065,001,647,753,652,567,1...",0ooo0ooo0.ooo
2902,order,0x3e6ab1b4b98dd45a1cd97fc141659077293095161497...,listing,"1,703,607,296.00","1,704,212,091.00",{'identifier': '117315147600270829665702926801...,1,,"{'quantity': '1000000000000000000', 'token_add...",1703607304,NaN,"11,731,514,760,027,082,503,362,540,473,080,970,...",tokentroopers.xyz


In [44]:
box_sales_os = sales[sales['name'].str.endswith('.box')]
box_bids_os = bids[bids['name'].str.endswith('.box')] 

In [45]:
box_bids_os.columns = [f'bid_{col}' if col != 'name' else col for col in box_bids_os.columns]

In [46]:
box_sales_os.columns = [f'sale_{col}' if col != 'name' else col for col in box_sales_os.columns]

In [47]:
box_bids_os

,bid_event_type,bid_order_hash,bid_order_type,bid_start_date,bid_expiration_date,bid_asset,bid_quantity,bid_taker,bid_payment,bid_event_timestamp,bid_transaction,bid_tokenid,name
0,order,0x49729029191017c544a3f67f03968dd12bf16c69e1dd...,listing,"1,720,562,975.00","1,736,460,564.00",{'identifier': '106968891900401063741132344938...,1,,"{'quantity': '33000000000000000000', 'token_ad...",1720563049,NaN,"106,968,891,900,401,057,912,681,673,593,283,028...",blum.box
1,order,0xaedefdd5ee2c58433e4135132b5b8454653725d40859...,listing,"1,720,561,057.00","1,721,165,912.00",{'identifier': '283604669650136775214415072033...,1,,"{'quantity': '1000000000000000000', 'token_add...",1720561062,NaN,"28,360,466,965,013,677,414,172,494,359,945,146,...",cypherpunk.box
2,order,0x9d15ad778c1865fa87e96f6fb85f90bed32d784c1162...,listing,"1,720,544,932.00","1,720,612,765.00",{'identifier': '112995818248598227413563035922...,1,,"{'quantity': '77700000000000000', 'token_addre...",1720544938,NaN,"112,995,818,248,598,233,060,911,981,365,146,698...",777777.box
3,order,0x3682a15b341d33dd93f859a3029596e346769720a6ea...,listing,"1,720,527,777.00","1,720,786,974.00",{'identifier': '277709650878091304863083034277...,1,,"{'quantity': '1690000000000000000', 'token_add...",1720527923,NaN,"27,770,965,087,809,130,203,304,471,643,289,691,...",ia.box
4,order,0xd76ea0933059e5d358e6768a87ac0358d4f7b6b3b77e...,listing,"1,720,527,863.00","1,723,206,142.00",{'identifier': '111468288816422208343542313783...,1,,"{'quantity': '960000000000000000', 'token_addr...",1720527874,NaN,"111,468,288,816,422,214,204,408,193,882,129,338...",girl.box
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2862,order,0x472882ed6ef6df3e646a968fe399a794cb2d80783c0d...,listing,"1,705,973,310.00","1,721,694,506.00",{'identifier': '103348240472784938693450072722...,1,,"{'quantity': '8000000000000000000', 'token_add...",1705973330,NaN,"103,348,240,472,784,936,049,086,084,648,101,639...",open.box
2863,order,0x9f91f3031bea58c65527094995a80c8ac04ec11cabc3...,listing,"1,705,951,814.00","1,706,211,009.00",{'identifier': '910130851175214067204502281907...,1,,"{'quantity': '9990000000000000000', 'token_add...",1705951959,NaN,"91,013,085,117,521,410,273,961,891,024,065,292,...",xxx.box
2865,order,0xf7eabd25103d87771cfa5b7ec1f833ff28b305e2e951...,listing,"1,705,880,300.00","1,706,485,096.00",{'identifier': '310600624953776604609951946603...,1,,"{'quantity': '18000000000000000000', 'token_ad...",1705880336,NaN,"31,060,062,495,377,660,754,331,643,768,291,496,...",88.box
2866,order,0x8a5a4f735475cfb1a1ad0d37ee3f6b550c731091ef1e...,listing,"1,705,876,048.00","1,721,600,844.00",{'identifier': '443446172444352102226461664950...,1,,"{'quantity': '88000000000000000000', 'token_ad...",1705876076,NaN,"44,344,617,244,435,210,889,725,244,831,655,211,...",o.box


In [48]:
box_listings_and_sales = pd.merge(box_bids_os, box_sales_os, how='inner', on='name')
box_listings_and_sales['bid_event_timestamp'] = pd.to_datetime(box_listings_and_sales['bid_event_timestamp'], unit='s')
box_listings_and_sales['sale_event_timestamp'] = pd.to_datetime(box_listings_and_sales['sale_event_timestamp'], unit='s')


In [49]:
filtered_box_listings_and_sales = box_listings_and_sales[box_listings_and_sales['sale_event_timestamp'] > box_listings_and_sales['bid_event_timestamp']]
filtered_box_listings_and_sales['time_diff'] = filtered_box_listings_and_sales['sale_event_timestamp'] - filtered_box_listings_and_sales['bid_event_timestamp']


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\2210088090.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_box_listings_and_sales['time_diff'] = filtered_box_listings_and_sales['sale_event_timestamp'] - filtered_box_listings_and_sales['bid_event_timestamp']


In [50]:
avg_time_to_sell = filtered_box_listings_and_sales['time_diff'].mean()
print(avg_time_to_sell)

19 days 11:19:25.506097561


In [51]:
closest_listings = filtered_box_listings_and_sales.loc[filtered_box_listings_and_sales.groupby(['name', 'sale_event_timestamp'])['time_diff'].idxmin()]


In [52]:
closest_listings['sale_quantity'] = closest_listings['sale_payment'].apply(lambda x: int(x['quantity']) / 10**18)
closest_listings['listing_quantity'] = closest_listings['bid_payment'].apply(lambda x: int(x['quantity']) / 10**18)


In [53]:
closest_listings[['bid_event_timestamp','listing_quantity']].head()

,bid_event_timestamp,listing_quantity
33,2024-04-23 10:22:52,0.03
68,2024-02-12 18:32:17,0.04
51,2024-02-24 06:41:03,0.04
64,2024-02-19 23:52:15,0.04
56,2024-02-23 04:00:29,0.10


In [54]:
closest_listings['bid_event_timestamp'] = closest_listings['bid_event_timestamp'].dt.strftime('%Y-%m-%d %H:00:00')


In [55]:
closest_listings['sale_event_timestamp'] = closest_listings['sale_event_timestamp'].dt.strftime('%Y-%m-%d %H:00:00')

In [56]:
mint_df['tokenid'] = mint_df['tokenid'].astype(float)

In [57]:
mints_with_names = pd.merge(mint_df, descriptions_df, how='left', on='tokenid')

In [58]:
mints_with_names

,day,tokenid,tx_hash,__row_index,name
0,2024-07-10T10:00:00.000Z,"8,350,705,726,613,897,402,252,190,637,230,342,5...",0x49c925349318bcbf089b910c7cbb9b75dcff8fd1b992...,0,None
1,2024-07-10T10:00:00.000Z,"8,350,705,726,613,897,402,252,190,637,230,342,5...",0x49c925349318bcbf089b910c7cbb9b75dcff8fd1b992...,1,None
2,2024-07-10T09:00:00.000Z,"831,581,688,049,173,449,027,930,491,313,066,372...",0x797479e07d791a0f79d0433a514dbb4fcd196e57a047...,2,None
3,2024-07-10T09:00:00.000Z,"831,581,688,049,173,449,027,930,491,313,066,372...",0x797479e07d791a0f79d0433a514dbb4fcd196e57a047...,3,None
4,2024-07-10T04:00:00.000Z,"50,886,887,701,499,123,067,873,961,026,079,613,...",0x4b19b2871109c40c80ae709a599edea386639794904e...,4,None
...,...,...,...,...,...
32817,2023-10-04T02:00:00.000Z,"73,079,797,870,479,156,483,046,577,349,357,812,...",0x981dd9c93e0419a227128b7ee95c620e7dbc88fe92e8...,32817,org
32818,2023-10-04T02:00:00.000Z,"97,049,678,982,547,095,836,606,605,282,336,114,...",0xd7ece008b97cad61f02396def394751c3fa9a4c2844a...,32818,net
32819,2023-10-04T02:00:00.000Z,"97,049,678,982,547,095,836,606,605,282,336,114,...",0xd7ece008b97cad61f02396def394751c3fa9a4c2844a...,32819,net
32820,2023-10-04T02:00:00.000Z,"77,875,674,875,312,083,922,572,966,307,662,148,...",0x64f70f46174e15001ff109ce325f9d0e0d7a571a22ed...,32820,com


In [59]:
mints_with_names_null = mints_with_names[mints_with_names.isnull().any(axis=1)]
print(list(mints_with_names_null['tx_hash']))

['0x49c925349318bcbf089b910c7cbb9b75dcff8fd1b992f6e53b76f785e13e801e', '0x49c925349318bcbf089b910c7cbb9b75dcff8fd1b992f6e53b76f785e13e801e', '0x797479e07d791a0f79d0433a514dbb4fcd196e57a047c701e768d61f4d6fc7c8', '0x797479e07d791a0f79d0433a514dbb4fcd196e57a047c701e768d61f4d6fc7c8', '0x4b19b2871109c40c80ae709a599edea386639794904ec26638a365efaf245965', '0x4b19b2871109c40c80ae709a599edea386639794904ec26638a365efaf245965', '0xb40404c76df837b074485f6a7e3f616f9554311199b769dbad5b0cb6efb1e6c8', '0xb40404c76df837b074485f6a7e3f616f9554311199b769dbad5b0cb6efb1e6c8', '0xaa9ce395c195046b86250dac7043d5a4e30170635d20c823de30ebc8625539b6', '0xaa9ce395c195046b86250dac7043d5a4e30170635d20c823de30ebc8625539b6', '0x38ae6b03b797a2e03b0ad126e3f1416996c406b2e1a50aa03c1ff4c19c25f1f2', '0x38ae6b03b797a2e03b0ad126e3f1416996c406b2e1a50aa03c1ff4c19c25f1f2', '0x90c71cda6d5b63fc80ac3acef0af808e86e44b3eaedf9a8daf8284428aa1a52a', '0x90c71cda6d5b63fc80ac3acef0af808e86e44b3eaedf9a8daf8284428aa1a52a', '0xb20eed88548ccb6f

In [60]:
mints_with_names.drop_duplicates('tokenid', inplace=True)

In [61]:
mints_with_names.drop(columns=['__row_index','tx_hash','tokenid'], inplace=True)

In [62]:
mints_with_names.set_index('day', inplace=True)

In [63]:
mints_with_names.index = pd.to_datetime(mints_with_names.index)
mints_with_names.dropna(inplace=True)

In [64]:
box_domains_mints = mints_with_names[mints_with_names['name'].str.endswith('.box')]


In [65]:
box_domains_mints

,name
day,
2024-07-09 06:00:00+00:00,helvetica.box
2024-07-09 06:00:00+00:00,gain.box
2024-07-09 06:00:00+00:00,0xsafe.box
2024-07-09 03:00:00+00:00,ons.box
2024-07-08 12:00:00+00:00,artists.box
...,...
2023-10-31 17:00:00+00:00,duncan.box
2023-10-26 14:00:00+00:00,intercap-dev.box
2023-10-24 19:00:00+00:00,s.box


In [66]:
daily_box_mints = box_domains_mints.resample('D').count()

In [67]:
daily_box_mints.rename(columns={'name':'mints'}, inplace=True)
daily_box_mints_fig = px.bar(daily_box_mints, x=daily_box_mints.index, y='mints', title='Daily Mints')
daily_box_mints_fig.show()

In [68]:
total_box_mints = box_domains_mints.count().iloc[0]
total_box_mints

np.int64(3613)

In [69]:
sales_with_names = pd.merge(sales_df, descriptions_df, how='left', on='tokenid')

In [70]:
sales_with_names.drop_duplicates('tokenid', inplace=True)
sales_with_names.drop(columns=['__row_index','tx_hash','tokenid'], inplace=True)
sales_with_names.set_index('day', inplace=True)
sales_with_names.index = pd.to_datetime(sales_with_names.index)

In [71]:
box_domains_sales = sales_with_names[sales_with_names['name'].str.endswith('.box')]
box_domains_sales

,price,name
day,,
2024-02-03 22:00:00+00:00,0.01,696.box
2024-02-05 04:00:00+00:00,0.02,song.box
2024-02-10 19:00:00+00:00,0.04,to.box
2024-02-20 12:00:00+00:00,0.07,uae.box
2024-02-22 04:00:00+00:00,0.04,08.box
2024-02-24 04:00:00+00:00,0.07,404.box
2024-02-25 07:00:00+00:00,0.10,38.box
2024-02-25 02:00:00+00:00,0.04,010.box
2024-02-25 03:00:00+00:00,0.04,ap.box


In [72]:
eth_usd_df.set_index('day', inplace=True)
eth_usd_df.index = pd.to_datetime(eth_usd_df.index)
eth_usd_df.drop(columns=['__row_index'], inplace=True)

In [73]:
eth_usd_df.rename(columns={'price':'eth_usd'}, inplace=True)

In [74]:
box_listings['start_time'] = pd.to_datetime(box_listings['start_time'], unit='s').dt.strftime('%Y-%m-%d %H:00:00')
box_listings['end_time'] = pd.to_datetime(box_listings['end_time'], unit='s').dt.strftime('%Y-%m-%d %H:00:00')

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\1480011625.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\1480011625.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [75]:
eth_usd_df_copy = eth_usd_df.reset_index().copy()
eth_usd_df_copy.rename(columns={'day':'start_time'}, inplace=True)

In [76]:
eth_usd_df_copy['start_time'] = pd.to_datetime(eth_usd_df_copy['start_time']).dt.tz_localize(None)

In [77]:
box_listings['start_time'] = pd.to_datetime(box_listings['start_time']) 

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\1939952577.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [78]:
eth_usd_df_copy_2 = eth_usd_df_copy.copy()
eth_usd_df_copy_2.rename(columns={'start_time':'bid_event_timestamp', 'eth_usd':'eth_usd_bid'}, inplace=True)

In [79]:
eth_usd_df_copy_3 = eth_usd_df_copy.copy()
eth_usd_df_copy_3.rename(columns={'start_time':'sale_event_timestamp', 'eth_usd':'eth_usd_sale'}, inplace=True)

In [80]:
closest_listings

,bid_event_type,bid_order_hash,bid_order_type,bid_start_date,bid_expiration_date,bid_asset,bid_quantity,bid_taker,bid_payment,bid_event_timestamp,...,sale_payment,sale_event_timestamp,sale_closing_date,sale_nft,sale_seller,sale_buyer,sale_transaction,sale_tokenid,time_diff,listing_quantity
33,order,0x9c93a66cfda75ee4644448fb58cae06eb0812df451fb...,listing,"1,713,867,753.00","1,716,459,753.00",{'identifier': '703498242317845484567858628589...,1,,"{'quantity': '29000000000000000', 'token_addre...",2024-04-23 10:00:00,...,"{'quantity': '29000000000000000', 'token_addre...",2024-04-24 04:00:00,"1,713,934,467.00",{'identifier': '703498242317845484567858628589...,0x0c289ec5d7fac13ecba85a404f144dfe461f6757,0x9f65a2c44f7d41da4a66cdb7a11cfc4d5c6a5701,0xa10ad4838da88ca27edb13cc4ad94b4f26d8c703a2e9...,"7,034,982,423,178,454,817,470,894,177,061,768,9...",0 days 18:31:35,0.03
68,order,0xae0f0119039a0405216a385824cce1662a72011f2bb4...,listing,"1,707,762,730.00","1,710,268,210.00",{'identifier': '496962274766683276127777586567...,1,,"{'quantity': '40000000000000000', 'token_addre...",2024-02-12 18:00:00,...,"{'quantity': '40000000000000000', 'token_addre...",2024-02-25 02:00:00,"1,708,829,917.00",{'identifier': '496962274766683276127777586567...,0x3276e82ebb1b4b9f01ab9286ed6bcc6603e368e2,0xf408bee3443d0397e2c1cde588fb060ac657006f,0x8f45ee1a6d326905bbc9ef018536ccda79a0b00543ad...,"49,696,227,476,668,324,726,036,518,805,210,474,...",12 days 08:26:20,0.04
51,order,0xb383acd8e244e0278b99e153f7a27d1a153020241595...,listing,"1,708,756,857.00","1,709,361,579.00",{'identifier': '255496510243247998967394616710...,1,,"{'quantity': '35000000000000000', 'token_addre...",2024-02-24 06:00:00,...,"{'quantity': '35000000000000000', 'token_addre...",2024-02-25 03:00:00,"1,708,832,065.00",{'identifier': '255496510243247998967394616710...,0xb7d8d3e6733e9e19887821e787297025a4fc93d3,0x9af5d20a9dffd92448dc0337b0900f3b48541de3,0x5285add55012e02f6d0705c017af6880a18b78777810...,"25,549,651,024,324,800,143,018,812,772,741,840,...",0 days 20:53:22,0.04
64,order,0x7c5dcfc329b68cf30adc1ae560f86b333ea00ac3c2d1...,listing,"1,708,386,727.00","1,710,888,605.00",{'identifier': '902676801938279802203143854774...,1,,"{'quantity': '40000000000000000', 'token_addre...",2024-02-19 23:00:00,...,"{'quantity': '40000000000000000', 'token_addre...",2024-02-22 04:00:00,"1,708,574,565.00",{'identifier': '902676801938279802203143854774...,0x6053904b1643182f235dd8f1a91e851662d93d35,0xa014bf15f0a6e208218c98c31f05338c45e4b010,0xcf386a26aae0dfebb8664c8d2c6590f67927d7aa4f7a...,"90,267,680,193,827,982,430,960,140,548,436,305,...",2 days 04:10:30,0.04
56,order,0x8a94cdc0c83ea1ba671a7b9bc041b3d39c32d31d7a71...,listing,"1,708,660,825.00","1,722,938,201.00",{'identifier': '110858464559300831902144923932...,1,,"{'quantity': '100000000000000000', 'token_addr...",2024-02-23 04:00:00,...,"{'quantity': '100000000000000000', 'token_addr...",2024-02-25 07:00:00,"1,708,846,421.00",{'identifier': '110858464559300831902144923932...,0x13918a208337f54005b8533b9d76d5b6e87303bb,0x27157c32c014598c7d743d8731e238ab151f2c65,0x77237a2ec2b775870ec09fc10310d399c80319a9b87e...,"110,858,464,559,300,825,990,569,710,147,236,992...",2 days 03:33:12,0.10
53,order,0x07afcb596b64f4420b0ffcf4bf57f171a6712d05e9ce...,listing,"1,708,746,396.00","1,724,471,192.00",{'identifier': '602749672470407216249937792665...,1,,"{'quantity': '70000000000000000', 'token_addre...",2024-02-24 03:00:00,...,"{'quantity': '70000000000000000', 'token_addre...",2024-02-25 03:00:00,"1,708,831,745.00",{'identifier': '602749672470407216249937792665...,0x4cfd50f62df880cccd5e6d489e9ea3039819aad1,0xf408bee3443d0397e2c1cde588fb060ac657006f,0x41efe31d90b0c7d45d028a1811d3beacd5e13ca768fe...,"60,274,967,247,040,719,711,675,361,166,867,087,...",0 days 23:42:22,0.07
52,order,0x36b9584c47a8999f3372488715cea97af37bcb61f248...,listing,"1,708,746,463.00","1,724,471,257.00",{'identifier': '200543934089042259451502567419...,1,,"{'quantity': '70000000000000000', 'toke

In [81]:
closest_listings['bid_event_timestamp'] = pd.to_datetime(closest_listings['bid_event_timestamp'])
closest_listings['sale_event_timestamp'] = pd.to_datetime(closest_listings['sale_event_timestamp'])

In [82]:
closest_listings = closest_listings.merge(eth_usd_df_copy_2, how='left', on='bid_event_timestamp')

In [83]:
closest_listings = closest_listings.merge(eth_usd_df_copy_3, how='left', on='sale_event_timestamp')

In [84]:
closest_listings['sale_usd'] = closest_listings['sale_quantity'] * closest_listings['eth_usd_sale']
closest_listings['list_usd'] = closest_listings['listing_quantity'] * closest_listings['eth_usd_bid']

In [85]:
closest_listings['percent_change'] = (closest_listings['sale_usd'] - closest_listings['list_usd']) / closest_listings['list_usd'] * 100

In [86]:
listing_price_to_sale_avg_pct_change = closest_listings['percent_change'].mean()
print(listing_price_to_sale_avg_pct_change)

-14.673687659116787


In [87]:

box_listings = box_listings.merge(eth_usd_df_copy, how='left', on='start_time') 

In [88]:
box_listings.drop(columns=['order_hash'], inplace=True)

In [89]:
box_listings['price_in_usd_start_time'] = box_listings['price_in_eth'] * box_listings['eth_usd']

In [90]:
box_listings.set_index('start_time', inplace=True)
box_listings_max_daily = box_listings['price_in_usd_start_time'].resample('D').max()

In [91]:
box_listings_num_daily = box_listings['name'].resample('D').count()
total_box_listings = box_listings_num_daily.sum()
total_box_listings

np.int64(529)

In [92]:
box_listings_num_daily

start_time
2024-01-23     3
2024-01-24    11
2024-01-25    11
2024-01-26     6
2024-01-27     4
              ..
2024-07-05     1
2024-07-06     7
2024-07-07     2
2024-07-08    23
2024-07-09     9
Freq: D, Name: name, Length: 169, dtype: int64

In [93]:
box_listings_min_daily = box_listings['price_in_usd_start_time'].resample('D').min()
box_listings_avg_daily = box_listings['price_in_usd_start_time'].resample('D').mean()

In [94]:
box_listings_max_daily.fillna(0, inplace=True)
box_listings_min_daily.fillna(0, inplace=True)
box_listings_avg_daily.fillna(0, inplace=True)

In [95]:
box_listing_data = pd.merge(box_listings_num_daily.to_frame('listings'), box_listings_max_daily.to_frame('max_price'), left_index=True,
                            right_index=True, how='inner')

In [96]:
box_listing_data = box_listing_data.merge(box_listings_min_daily.to_frame('min_price'), left_index=True,
                                          right_index=True, how='inner')

box_listing_data = box_listing_data.merge(box_listings_avg_daily.to_frame('avg_price'), left_index=True,
                                          right_index=True, how='inner')

In [97]:
box_listing_data

,listings,max_price,min_price,avg_price
start_time,,,,
2024-01-23,3,"18,508.01","6,582.45","10,557.64"
2024-01-24,11,"931,166.04","3,349.20","160,128.32"
2024-01-25,11,"2,206,503.58",660.15,"882,610.90"
2024-01-26,6,"665,376.23","4,478.13","162,767.74"
2024-01-27,4,"1,590,895.35","680,521.15","946,259.95"
...,...,...,...,...
2024-07-05,1,"8,581.74","8,581.74","8,581.74"
2024-07-06,7,"30,690.04","23,780.08","25,754.35"
2024-07-07,2,"39,241.41","9,010.05","24,125.73"


In [98]:
monthly_listings = box_listings['name'].resample('M').count()
monthly_listings

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\1847853540.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



start_time
2024-01-31     36
2024-02-29     17
2024-03-31     11
2024-04-30     93
2024-05-31    144
2024-06-30    173
2024-07-31     55
Freq: ME, Name: name, dtype: int64

In [99]:
def monthly_listings_growth_rate(listings):
    previous_month = listings.shift(1)
    growth_rate = ((listings - previous_month) / previous_month) * 100
    return growth_rate

In [100]:
listings_growth_rate = monthly_listings_growth_rate(monthly_listings)


In [101]:
listings_growth_rate.dropna(inplace=True)
listings_growth_rate

start_time
2024-02-29   -52.78
2024-03-31   -35.29
2024-04-30   745.45
2024-05-31    54.84
2024-06-30    20.14
2024-07-31   -68.21
Freq: ME, Name: name, dtype: float64

In [102]:
box_domains_sales = box_domains_sales.merge(eth_usd_df, left_index=True, right_index=True, how='left')

In [103]:
box_domains_sales['price_usd'] = box_domains_sales['price'] * box_domains_sales['eth_usd']
box_domains_sales.rename(columns={'price':'price_eth'}, inplace=True)

In [104]:
box_domains_sales.drop(columns=['eth_usd'], inplace=True)


In [105]:
box_domains_sales.sort_index(inplace=True)
box_domains_mints.sort_index(inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\3245620862.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [106]:
box_domains_sales = box_domains_sales[['name', 'price_usd','price_eth']]


In [107]:
box_domains_sales

,name,price_usd,price_eth
day,,,
2024-02-03 22:00:00+00:00,696.box,33.09,0.01
2024-02-05 04:00:00+00:00,song.box,45.60,0.02
2024-02-10 19:00:00+00:00,to.box,96.97,0.04
2024-02-20 12:00:00+00:00,uae.box,203.19,0.07
2024-02-22 04:00:00+00:00,08.box,116.74,0.04
2024-02-24 04:00:00+00:00,404.box,203.68,0.07
2024-02-25 02:00:00+00:00,010.box,119.08,0.04
2024-02-25 03:00:00+00:00,75.box,269.17,0.09
2024-02-25 03:00:00+00:00,015.box,104.68,0.04


In [108]:
max_eth_sale = box_domains_sales['price_eth'].max()
max_usd_sale = box_domains_sales['price_usd'].max()

# Retrieve the corresponding timestamps
max_eth_sale_row = box_domains_sales.loc[box_domains_sales['price_eth'].idxmax()]
max_usd_sale_row = box_domains_sales.loc[box_domains_sales['price_usd'].idxmax()]

# Display the results
print(f"Maximum sale: \n {max_eth_sale_row}")


Maximum sale: 
 name           vm.box
price_usd   31,481.47
price_eth       10.00
Name: 2024-04-27 16:00:00+00:00, dtype: object


In [109]:
total_box_sales = box_domains_sales['name'].count()
print(f'total .box domain sales as of {dt.datetime.today()} : {total_box_sales}')

total .box domain sales as of 2024-07-10 11:05:41.508139 : 21


In [110]:
daily_box_sales = box_domains_sales['name'].resample('D').count()
daily_box_sales

day
2024-02-03 00:00:00+00:00    1
2024-02-04 00:00:00+00:00    0
2024-02-05 00:00:00+00:00    1
2024-02-06 00:00:00+00:00    0
2024-02-07 00:00:00+00:00    0
                            ..
2024-05-03 00:00:00+00:00    0
2024-05-04 00:00:00+00:00    0
2024-05-05 00:00:00+00:00    0
2024-05-06 00:00:00+00:00    0
2024-05-07 00:00:00+00:00    1
Freq: D, Name: name, Length: 95, dtype: int64

In [111]:
daily_box_vol = box_domains_sales['price_usd'].resample('D').sum()
cumulative_box_vol = daily_box_vol.cumsum()
cumulative_box_vol

day
2024-02-03 00:00:00+00:00       33.09
2024-02-04 00:00:00+00:00       33.09
2024-02-05 00:00:00+00:00       78.69
2024-02-06 00:00:00+00:00       78.69
2024-02-07 00:00:00+00:00       78.69
                               ...   
2024-05-03 00:00:00+00:00   35,720.31
2024-05-04 00:00:00+00:00   35,720.31
2024-05-05 00:00:00+00:00   35,720.31
2024-05-06 00:00:00+00:00   35,720.31
2024-05-07 00:00:00+00:00   35,888.14
Freq: D, Name: price_usd, Length: 95, dtype: float64

In [112]:
daily_box_sales_fig = px.bar(daily_box_sales.to_frame('sales'), x=daily_box_sales.index, y='sales', title='Daily Sales')
daily_box_sales_fig.show()

In [113]:
latest_box_domains_sales = box_domains_sales.iloc[-10:] 
latest_box_domains_sales

,name,price_usd,price_eth
day,,,
2024-02-25 03:00:00+00:00,40.box,209.36,0.07
2024-02-25 07:00:00+00:00,38.box,301.73,0.10
2024-04-17 18:00:00+00:00,mewtwo.box,29.61,0.01
2024-04-24 04:00:00+00:00,0000.box,93.92,0.03
2024-04-25 21:00:00+00:00,express.box,317.33,0.10
2024-04-27 00:00:00+00:00,jane.box,"1,151.71",0.37
2024-04-27 16:00:00+00:00,vm.box,"31,481.47",10.00
2024-04-28 01:00:00+00:00,man.box,224.60,0.07
2024-04-28 07:00:00+00:00,good.box,314.61,0.10


In [114]:
latest_box_domains_mints = box_domains_mints.iloc[-10:]
latest_box_domains_mints

,name
day,
2024-07-07 02:00:00+00:00,contract.box
2024-07-07 02:00:00+00:00,mt.box
2024-07-07 14:00:00+00:00,pov.box
2024-07-07 18:00:00+00:00,university.box
2024-07-07 21:00:00+00:00,pharmacy.box
2024-07-08 12:00:00+00:00,artists.box
2024-07-09 03:00:00+00:00,ons.box
2024-07-09 06:00:00+00:00,0xsafe.box
2024-07-09 06:00:00+00:00,gain.box


In [115]:
cumulative_box_mints = daily_box_mints.cumsum()
cumulative_box_mints.rename(columns={'mints':'cumulative mints'}, inplace=True)

In [116]:
daily_mint_metrics = daily_box_mints.merge(cumulative_box_mints, left_index=True, right_index=True, how='inner')
daily_mint_metrics

,mints,cumulative mints
day,,
2023-10-04 00:00:00+00:00,1,1
2023-10-05 00:00:00+00:00,0,1
2023-10-06 00:00:00+00:00,0,1
2023-10-07 00:00:00+00:00,0,1
2023-10-08 00:00:00+00:00,0,1
...,...,...
2024-07-05 00:00:00+00:00,5,3599
2024-07-06 00:00:00+00:00,3,3602
2024-07-07 00:00:00+00:00,6,3608


In [117]:
daily_mint_metrics_fig = make_subplots(specs=[[{"secondary_y": True}]])

daily_mint_metrics_fig.add_trace(
    go.Bar(
        x=daily_mint_metrics.index,
        y=daily_mint_metrics['mints'],
        name='Mints'
    ),
    secondary_y=False
)

daily_mint_metrics_fig.add_trace(
    go.Scatter(
        x=daily_mint_metrics.index,
        y=daily_mint_metrics['cumulative mints'],
        name='Cumulative Mints',
        mode='lines'
    ),
    secondary_y=True
)

daily_mint_metrics_fig.update_xaxes(title_text="Date")

daily_mint_metrics_fig.show()


In [118]:
listings_growth_rate

start_time
2024-02-29   -52.78
2024-03-31   -35.29
2024-04-30   745.45
2024-05-31    54.84
2024-06-30    20.14
2024-07-31   -68.21
Freq: ME, Name: name, dtype: float64

In [119]:
listings_growth_rate_fig = px.bar(listings_growth_rate.to_frame('Monthly Listings Growth Rate'), x=listings_growth_rate.index,
                                   y='Monthly Listings Growth Rate', title='Monthly Listings Growth Rate')

listings_growth_rate_fig.show()

In [120]:
cumulative_box_sales = daily_box_sales.cumsum()
cumulative_box_sales

day
2024-02-03 00:00:00+00:00     1
2024-02-04 00:00:00+00:00     1
2024-02-05 00:00:00+00:00     2
2024-02-06 00:00:00+00:00     2
2024-02-07 00:00:00+00:00     2
                             ..
2024-05-03 00:00:00+00:00    20
2024-05-04 00:00:00+00:00    20
2024-05-05 00:00:00+00:00    20
2024-05-06 00:00:00+00:00    20
2024-05-07 00:00:00+00:00    21
Freq: D, Name: name, Length: 95, dtype: int64

In [121]:
monthly_max_sold = box_domains_sales['price_usd'].resample('M').max()
monthly_min_sold = box_domains_sales['price_usd'].resample('M').min()
monthly_avg_sold = box_domains_sales['price_usd'].resample('M').mean()
monthly_volume_usd = box_domains_sales['price_usd'].resample('M').sum()
monthly_num_sold = box_domains_sales['name'].resample('M').count()

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\3455025639.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\3455025639.py:2: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\3455025639.py:3: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\3455025639.py:4: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\3455025639.py:5: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [122]:
monthly_max_sold.fillna(0, inplace=True)
monthly_min_sold.fillna(0, inplace=True)
monthly_avg_sold.fillna(0, inplace=True)
monthly_volume_usd.fillna(0, inplace=True)
monthly_num_sold.fillna(0, inplace=True)

In [123]:
monthly_box_sales_metrics = pd.merge(monthly_max_sold.to_frame('max_price'), monthly_min_sold.to_frame('min_price'), left_index=True, right_index=True, how='inner')

In [124]:
monthly_box_sales_metrics = monthly_box_sales_metrics.merge(monthly_avg_sold.to_frame('avg_price'), left_index=True, right_index=True, how='inner')

In [125]:
monthly_box_sales_metrics = monthly_box_sales_metrics.merge(monthly_volume_usd.to_frame('volume_usd'), left_index=True, right_index=True, how='inner')

In [126]:
monthly_num_sold

day
2024-02-29 00:00:00+00:00    13
2024-03-31 00:00:00+00:00     0
2024-04-30 00:00:00+00:00     7
2024-05-31 00:00:00+00:00     1
Freq: ME, Name: name, dtype: int64

In [127]:
monthly_box_sales_metrics = monthly_box_sales_metrics.merge(monthly_num_sold.to_frame('domains sold'), left_index=True, right_index=True, how='inner')

In [128]:
monthly_box_sales_metrics

,max_price,min_price,avg_price,volume_usd,domains sold
day,,,,,
2024-02-29 00:00:00+00:00,301.73,33.09,162.08,"2,107.05",13
2024-03-31 00:00:00+00:00,0.00,0.00,0.00,0.00,0
2024-04-30 00:00:00+00:00,"31,481.47",29.61,"4,801.89","33,613.26",7
2024-05-31 00:00:00+00:00,167.83,167.83,167.83,167.83,1


In [129]:
daily_sales_metrics = pd.merge(cumulative_box_sales.to_frame('cumulative_sales'), daily_box_sales.to_frame('daily_sales'), 
                               left_index=True, right_index=True, how='left')

In [130]:
daily_sales_metrics = daily_sales_metrics.merge(daily_box_vol.to_frame('vol_usd'), left_index=True, right_index=True, how='inner')
daily_sales_metrics = daily_sales_metrics.merge(cumulative_box_vol.to_frame('cumulative_vol'), left_index=True, right_index=True, how='inner')

In [131]:
daily_sales_metrics

,cumulative_sales,daily_sales,vol_usd,cumulative_vol
day,,,,
2024-02-03 00:00:00+00:00,1,1,33.09,33.09
2024-02-04 00:00:00+00:00,1,0,0.00,33.09
2024-02-05 00:00:00+00:00,2,1,45.60,78.69
2024-02-06 00:00:00+00:00,2,0,0.00,78.69
2024-02-07 00:00:00+00:00,2,0,0.00,78.69
...,...,...,...,...
2024-05-03 00:00:00+00:00,20,0,0.00,"35,720.31"
2024-05-04 00:00:00+00:00,20,0,0.00,"35,720.31"
2024-05-05 00:00:00+00:00,20,0,0.00,"35,720.31"


In [132]:
daily_vol_fig = make_subplots(specs=[[{"secondary_y": True}]])

daily_vol_fig.add_trace(
    go.Bar(
        x=daily_sales_metrics.index,
        y=daily_sales_metrics['vol_usd'],
        name='Sales Volume'
    ),
    secondary_y=False
)

daily_vol_fig.add_trace(
    go.Scatter(
        x=daily_sales_metrics.index,
        y=daily_sales_metrics['cumulative_vol'],
        name='Cumulative Sales Volume',
        mode='lines'
    ),
    secondary_y=True
)

daily_vol_fig.update_xaxes(title_text="Date")

daily_vol_fig.show()


In [133]:
daily_sales_fig = make_subplots(specs=[[{"secondary_y": True}]])

daily_sales_fig.add_trace(
    go.Bar(
        x=daily_sales_metrics.index,
        y=daily_sales_metrics['daily_sales'],
        name='Sales'
    ),
    secondary_y=False
)

daily_sales_fig.add_trace(
    go.Scatter(
        x=daily_sales_metrics.index,
        y=daily_sales_metrics['cumulative_sales'],
        name='Cumulative Sales',
        mode='lines'
    ),
    secondary_y=True
)

daily_sales_fig.update_xaxes(title_text="Date")

daily_sales_fig.show()


In [134]:
monthly_listings = monthly_listings.to_frame('listings')


In [135]:
monthly_listings

,listings
start_time,
2024-01-31,36
2024-02-29,17
2024-03-31,11
2024-04-30,93
2024-05-31,144
2024-06-30,173
2024-07-31,55


In [136]:
monthly_sales = box_domains_sales['name'].resample('M').count()
monthly_sales = monthly_sales.reset_index()
monthly_sales['day'] = pd.to_datetime(monthly_sales['day']).dt.strftime('%Y-%m-%d')
monthly_sales.set_index('day', inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\2388482189.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [137]:
monthly_sales

,name
day,
2024-02-29,13
2024-03-31,0
2024-04-30,7
2024-05-31,1


In [138]:
monthly_listings.index

DatetimeIndex(['2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30', '2024-07-31'],
              dtype='datetime64[ns]', name='start_time', freq='ME')

In [139]:
monthly_sales.index = pd.to_datetime(monthly_sales.index)

In [140]:
monthly_sales_reindexed = monthly_sales.reindex(monthly_listings.index).fillna(0)
monthly_sales_reindexed


,name
start_time,
2024-01-31,0.00
2024-02-29,13.00
2024-03-31,0.00
2024-04-30,7.00
2024-05-31,1.00
2024-06-30,0.00
2024-07-31,0.00


In [141]:
monthly_listings['sales'] = monthly_sales_reindexed['name']

In [142]:
monthly_listings

,listings,sales
start_time,,
2024-01-31,36,0.00
2024-02-29,17,13.00
2024-03-31,11,0.00
2024-04-30,93,7.00
2024-05-31,144,1.00
2024-06-30,173,0.00
2024-07-31,55,0.00


In [143]:
monthly_listings['listings_to_sales_ratio'] = monthly_listings['listings'] / monthly_listings['sales']
monthly_listings['listings_to_sales_ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)

monthly_listings

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\3641184892.py:2: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





,listings,sales,listings_to_sales_ratio
start_time,,,
2024-01-31,36,0.00,0.00
2024-02-29,17,13.00,1.31
2024-03-31,11,0.00,0.00
2024-04-30,93,7.00,13.29
2024-05-31,144,1.00,144.00
2024-06-30,173,0.00,0.00
2024-07-31,55,0.00,0.00


In [144]:
listing_to_sales_fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar chart for sales
listing_to_sales_fig.add_trace(
    go.Bar(
        x=monthly_listings.index,
        y=monthly_listings['sales'],
        name='Sales'
    ),
    secondary_y=False
)

listing_to_sales_fig.add_trace(
    go.Bar(
        x=monthly_listings.index,
        y=monthly_listings['listings'],
        name='Listings'
    ),
    secondary_y=False
)

# Add line chart for cumulative sales
listing_to_sales_fig.add_trace(
    go.Scatter(
        x=monthly_listings.index,
        y=monthly_listings['listings_to_sales_ratio'],
        name='Listings to Sales Ratio',
        mode='lines'
    ),
    secondary_y=True
)

listing_to_sales_fig.show()


In [145]:
monthly_mints = box_domains_mints.resample('M').count()
monthly_mints.reset_index(inplace=True)
monthly_mints['day'] = pd.to_datetime(monthly_mints['day']).dt.strftime('%Y-%m-%d') 
print(monthly_mints)

          day  name
0  2023-10-31     5
1  2023-11-30     2
2  2023-12-31    23
3  2024-01-31  1745
4  2024-02-29   349
5  2024-03-31   162
6  2024-04-30   636
7  2024-05-31   414
8  2024-06-30   221
9  2024-07-31    56


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\2186263740.py:1: FutureWarning:

'M' is deprecated and will be removed in a future version, please use 'ME' instead.



In [146]:
monthly_mints.set_index('day', inplace=True)


In [147]:
monthly_mints.index = pd.to_datetime(monthly_mints.index) 
monthly_mints.index

DatetimeIndex(['2023-10-31', '2023-11-30', '2023-12-31', '2024-01-31',
               '2024-02-29', '2024-03-31', '2024-04-30', '2024-05-31',
               '2024-06-30', '2024-07-31'],
              dtype='datetime64[ns]', name='day', freq=None)

In [148]:
monthly_sales_reindexed.index

DatetimeIndex(['2024-01-31', '2024-02-29', '2024-03-31', '2024-04-30',
               '2024-05-31', '2024-06-30', '2024-07-31'],
              dtype='datetime64[ns]', name='start_time', freq='ME')

In [149]:
monthly_mints['sales'] = monthly_sales_reindexed['name'] 
monthly_mints.fillna(0, inplace=True)
monthly_mints['mint_to_sales_ratio'] = monthly_mints['name'] / monthly_mints['sales'] 
monthly_mints['mint_to_sales_ratio'].replace([float('inf'), -float('inf')], 0, inplace=True)
monthly_mints.rename(columns={'name':'mints'}, inplace=True)


C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\760772224.py:4: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [150]:
monthly_mints

,mints,sales,mint_to_sales_ratio
day,,,
2023-10-31,5,0.00,0.00
2023-11-30,2,0.00,0.00
2023-12-31,23,0.00,0.00
2024-01-31,1745,0.00,0.00
2024-02-29,349,13.00,26.85
2024-03-31,162,0.00,0.00
2024-04-30,636,7.00,90.86
2024-05-31,414,1.00,414.00
2024-06-30,221,0.00,0.00


In [151]:
mint_to_sales_fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar chart for sales
mint_to_sales_fig.add_trace(
    go.Bar(
        x=monthly_mints.index,
        y=monthly_mints['sales'],
        name='Sales'
    ),
    secondary_y=False
)

mint_to_sales_fig.add_trace(
    go.Bar(
        x=monthly_mints.index,
        y=monthly_mints['mints'],
        name='Mints'
    ),
    secondary_y=False
)

# Add line chart for cumulative sales
mint_to_sales_fig.add_trace(
    go.Scatter(
        x=monthly_mints.index,
        y=monthly_mints['mint_to_sales_ratio'],
        name='Mints to Sales Ratio',
        mode='lines'
    ),
    secondary_y=True
)

mint_to_sales_fig.show()


In [152]:
cumulative_listings_to_sales = total_box_listings / total_box_sales
print(cumulative_listings_to_sales)

25.19047619047619


In [153]:
cumulative_mint_to_sales = total_box_mints / total_box_sales
print(cumulative_mint_to_sales)

172.04761904761904


In [154]:
import os
print(os.getcwd())


e:\Projects\box_app


***Box Domains Valuation Model***

**Data Processing**

domain_path = 'E:/Projects/box_app/data/domain-name-sales.tsv'  
domain_data = pd.read_csv(domain_path, delimiter='\t')

In [155]:
domain_path = 'data/domain-name-sales.tsv'  
domain_data = pd.read_csv(domain_path, delimiter='\t')


In [156]:
domain_data.set_index('date', inplace=True)
domain_data = domain_data.drop(columns=['venue'])
domain_data.sort_index(inplace=True)

In [157]:
domain_data.index = pd.to_datetime(domain_data.index)
domain_data

,domain,price
date,,
1999-04-01,altavista.com,3250000
1999-04-01,bingo.com,1100000
1999-11-01,fly.com,1500000
1999-12-01,autos.com,2200000
1999-12-01,england.com,2000000
...,...,...
2021-01-01,yes.movie,253
2021-01-01,yopal.com,405
2021-01-01,yougraph.com,1161


In [158]:
domain_data['domain_length'] = domain_data['domain'].apply(len)
domain_data['num_vowels'] = domain_data['domain'].apply(lambda x: sum([1 for char in x if char in 'aeiou']))
domain_data['num_consonants'] = domain_data['domain'].apply(lambda x: sum([1 for char in x if char.isalpha() and char not in 'aeiou']))
domain_data['tld'] = domain_data['domain'].apply(lambda x: x.split('.')[-1])  # Extract TLD


In [159]:
domain_data

,domain,price,domain_length,num_vowels,num_consonants,tld
date,,,,,,
1999-04-01,altavista.com,3250000,13,5,7,com
1999-04-01,bingo.com,1100000,9,3,5,com
1999-11-01,fly.com,1500000,7,1,5,com
1999-12-01,autos.com,2200000,9,4,4,com
1999-12-01,england.com,2000000,11,3,7,com
...,...,...,...,...,...,...
2021-01-01,yes.movie,253,9,4,4,movie
2021-01-01,yopal.com,405,9,3,5,com
2021-01-01,yougraph.com,1161,12,4,7,com


In [160]:
box_domains_sales.columns

Index(['name', 'price_usd', 'price_eth'], dtype='object')

In [161]:
filtered_box = box_domains_sales.drop(columns=['price_eth'])
filtered_box.rename(columns={'name':'domain', 'price_usd':'price'}, inplace=True)


In [162]:
filtered_box['domain_length'] = filtered_box['domain'].apply(len)
filtered_box['num_vowels'] = filtered_box['domain'].apply(lambda x: sum([1 for char in x if char in 'aeiou']))
filtered_box['num_consonants'] = filtered_box['domain'].apply(lambda x: sum([1 for char in x if char.isalpha() and char not in 'aeiou']))
filtered_box['tld'] = filtered_box['domain'].apply(lambda x: x.split('.')[-1])  # Extract TLD


In [163]:
filtered_box.index = filtered_box.index.strftime('%Y-%m-%d')

In [164]:
filtered_box

,domain,price,domain_length,num_vowels,num_consonants,tld
day,,,,,,
2024-02-03,696.box,33.09,7,1,2,box
2024-02-05,song.box,45.60,8,2,5,box
2024-02-10,to.box,96.97,6,2,3,box
2024-02-20,uae.box,203.19,7,4,2,box
2024-02-22,08.box,116.74,6,1,2,box
2024-02-24,404.box,203.68,7,1,2,box
2024-02-25,010.box,119.08,7,1,2,box
2024-02-25,75.box,269.17,6,1,2,box
2024-02-25,015.box,104.68,7,1,2,box


In [165]:
features = ['domain_length', 'num_vowels', 'num_consonants', 'tld']
X = domain_data[features]
y = domain_data['price']

In [166]:
# Preprocess categorical data (TLD) and handle missing values
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), ['domain_length', 'num_vowels', 'num_consonants']),
        ('cat', Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), ['tld'])
    ]
)

# Create a pipeline with Ridge regression
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge())
])

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'regressor__alpha': [0.1, 1.0, 10.0, 100.0, 1000.0]
}

In [167]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


**Ridge Regression**

In [168]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='r2')
grid_search.fit(X_train, y_train)

# Best model from grid search
best_model = grid_search.best_estimator_

# Predict and evaluate
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'Best Alpha: {grid_search.best_params_["regressor__alpha"]}')
print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

Best Alpha: 1000.0
MAE: 2705.0255351946857
MSE: 2852191402.3463864
R²: 0.0003925999206353392


**Random Forest Regressor**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=200, max_depth=20, min_samples_split=5, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**XGBoost**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**LightGBM**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LGBMRegressor(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**Cat Boost**

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', CatBoostRegressor(iterations=200, depth=5, learning_rate=0.1, random_state=42, verbose=0))
])

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

**Prophet**

from sklearn.base import BaseEstimator, TransformerMixin

class ProphetRegressor(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.model = Prophet()
        self.fitted_model = None

    def fit(self, X, y=None):
        df = pd.DataFrame({'ds': X.squeeze(), 'y': y})
        self.fitted_model = self.model.fit(df)
        return self

    def predict(self, X):
        future = pd.DataFrame({'ds': X.squeeze()})
        forecast = self.fitted_model.predict(future)
        return forecast['yhat'].values

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', ProphetRegressor())
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



**Best Model**

In [169]:
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', Ridge(alpha=1000.0))  # Set the best alpha value from grid search
])

In [170]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

MAE: 2705.0255351946857
MSE: 2852191402.3463864
R²: 0.0003925999206353392


In [171]:
filtered_box

,domain,price,domain_length,num_vowels,num_consonants,tld
day,,,,,,
2024-02-03,696.box,33.09,7,1,2,box
2024-02-05,song.box,45.60,8,2,5,box
2024-02-10,to.box,96.97,6,2,3,box
2024-02-20,uae.box,203.19,7,4,2,box
2024-02-22,08.box,116.74,6,1,2,box
2024-02-24,404.box,203.68,7,1,2,box
2024-02-25,010.box,119.08,7,1,2,box
2024-02-25,75.box,269.17,6,1,2,box
2024-02-25,015.box,104.68,7,1,2,box


In [172]:
box_X = filtered_box[features]

# Predict prices for .box domains using the best model
filtered_box['predicted_price'] = pipeline.predict(box_X)

print(filtered_box[['domain', 'predicted_price']])

                 domain  predicted_price
day                                     
2024-02-03      696.box         1,168.93
2024-02-05     song.box         2,173.63
2024-02-10       to.box         3,066.39
2024-02-20      uae.box         3,180.12
2024-02-22       08.box         2,005.65
2024-02-24      404.box         1,168.93
2024-02-25      010.box         1,168.93
2024-02-25       75.box         2,005.65
2024-02-25      015.box         1,168.93
2024-02-25       70.box         2,005.65
2024-02-25       ap.box         3,066.39
2024-02-25       40.box         2,005.65
2024-02-25       38.box         2,005.65
2024-04-17   mewtwo.box         1,560.93
2024-04-24     0000.box           332.21
2024-04-25  express.box         1,114.55
2024-04-27     jane.box         2,453.69
2024-04-27       vm.box         2,786.34
2024-04-28      man.box         2,620.01
2024-04-28     good.box         2,453.69
2024-05-07     onyx.box         2,173.63


In [173]:
r2 = r2_score(filtered_box['price'], filtered_box['predicted_price'])
print(f'r2 {r2}')

r2 0.0411345241547072


**.Box Domain Valuator**

In [174]:
filtered_box_2 = filtered_box.drop(columns=['predicted_price'])
filtered_box_2

,domain,price,domain_length,num_vowels,num_consonants,tld
day,,,,,,
2024-02-03,696.box,33.09,7,1,2,box
2024-02-05,song.box,45.60,8,2,5,box
2024-02-10,to.box,96.97,6,2,3,box
2024-02-20,uae.box,203.19,7,4,2,box
2024-02-22,08.box,116.74,6,1,2,box
2024-02-24,404.box,203.68,7,1,2,box
2024-02-25,010.box,119.08,7,1,2,box
2024-02-25,75.box,269.17,6,1,2,box
2024-02-25,015.box,104.68,7,1,2,box


In [175]:
combined_data = pd.concat([domain_data, filtered_box_2], ignore_index=True)

In [176]:
combined_data

,domain,price,domain_length,num_vowels,num_consonants,tld
0,altavista.com,"3,250,000.00",13,5,7,com
1,bingo.com,"1,100,000.00",9,3,5,com
2,fly.com,"1,500,000.00",7,1,5,com
3,autos.com,"2,200,000.00",9,4,4,com
4,england.com,"2,000,000.00",11,3,7,com
...,...,...,...,...,...,...
348252,jane.box,"1,151.71",8,3,4,box
348253,vm.box,"31,481.47",6,1,4,box
348254,man.box,224.60,7,2,4,box
348255,good.box,314.61,8,3,4,box


In [177]:
X = combined_data[features]
y = combined_data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')


MAE: 2708.9834581005175
MSE: 2845754262.680389
R²: 0.00041631417325493913


In [178]:
def model_prep(data):
    data['domain_length'] = data['domain'].apply(len)
    data['num_vowels'] = data['domain'].apply(lambda x: sum([1 for char in x if char in 'aeiou']))
    data['num_consonants'] = data['domain'].apply(lambda x: sum([1 for char in x if char.isalpha() and char not in 'aeiou']))
    data['tld'] = data['domain'].apply(lambda x: x.split('.')[-1]) 
    return data

In [179]:
def value_domain(domain):
    domain_x = domain[features]
    value = pipeline.predict(domain_x)
    print(f'predicted value: {value[0]}')
    return value[0] 

In [180]:
test_domain = 'eth.box' ## for model, just have the person input before .box, have it automatically add .box
test_domain_df = pd.DataFrame({'domain': [test_domain]})
test_domain_processed = model_prep(test_domain_df)
test_domain_value = value_domain(test_domain_processed)

predicted value: 2560.0528806059547


In [181]:
test_domain_value

np.float64(2560.0528806059547)

***Dash App***

Domain valuator would be callback

In [182]:
help(dcc)

Help on package dash.dcc in dash:

NAME
    dash.dcc

PACKAGE CONTENTS
    Checklist
    Clipboard
    ConfirmDialog
    ConfirmDialogProvider
    DatePickerRange
    DatePickerSingle
    Download
    Dropdown
    Geolocation
    Graph
    Input
    Interval
    Link
    Loading
    Location
    LogoutButton
    Markdown
    RadioItems
    RangeSlider
    Slider
    Store
    Tab
    Tabs
    Textarea
    Tooltip
    Upload
    _imports_
    express

CLASSES
    dash.development.base_component.Component(builtins.object)
        dash.dcc.Checklist.Checklist
        dash.dcc.Clipboard.Clipboard
        dash.dcc.ConfirmDialog.ConfirmDialog
        dash.dcc.ConfirmDialogProvider.ConfirmDialogProvider
        dash.dcc.DatePickerRange.DatePickerRange
        dash.dcc.DatePickerSingle.DatePickerSingle
        dash.dcc.Download.Download
        dash.dcc.Dropdown.Dropdown
        dash.dcc.Geolocation.Geolocation
        dash.dcc.Graph.Graph
        dash.dcc.Input.Input
        dash.dcc.Interval

In [183]:
latest_box_domains_sales.reset_index(inplace=True)

In [184]:
latest_box_domains_sales.sort_values(by='day', ascending=False, inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\3336277017.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [185]:
latest_box_domains_mints.reset_index(inplace=True)

In [186]:
latest_box_domains_mints.sort_values(by='day', ascending=False, inplace=True)

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\4287417152.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [187]:
avg_box_sale = box_domains_sales['price_usd'].mean()

In [188]:
highest_selling_domains = box_domains_sales[['name','price_usd','price_eth']].sort_values(by='price_usd', ascending=False)
highest_selling_domains = highest_selling_domains.head(10)
highest_selling_domains

,name,price_usd,price_eth
day,,,
2024-04-27 16:00:00+00:00,vm.box,"31,481.47",10.00
2024-04-27 00:00:00+00:00,jane.box,"1,151.71",0.37
2024-04-25 21:00:00+00:00,express.box,317.33,0.10
2024-04-28 07:00:00+00:00,good.box,314.61,0.10
2024-02-25 07:00:00+00:00,38.box,301.73,0.10
2024-02-25 03:00:00+00:00,70.box,269.17,0.09
2024-02-25 03:00:00+00:00,75.box,269.17,0.09
2024-04-28 01:00:00+00:00,man.box,224.60,0.07
2024-02-25 03:00:00+00:00,40.box,209.36,0.07


In [189]:
monthly_box_sales_metrics['cumulative_volume'] = monthly_box_sales_metrics['volume_usd'].cumsum()
monthly_box_sales_metrics.reset_index(inplace=True) 
monthly_box_sales_metrics.sort_values(by='day', ascending=False, inplace=True)

In [190]:
monthly_box_sales_metrics['cumulative domains sold'] = monthly_box_sales_metrics['domains sold'].cumsum()

In [191]:
historical_listing_to_sales = closest_listings[['name','bid_event_timestamp','sale_event_timestamp',
                                                'list_usd','sale_usd','percent_change']]
historical_listing_to_sales.sort_values(by='bid_event_timestamp', ascending=False, inplace=True)
historical_listing_to_sales

C:\Users\brandyns pc\AppData\Local\Temp\ipykernel_51256\2820302304.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,name,bid_event_timestamp,sale_event_timestamp,list_usd,sale_usd,percent_change
23,r.box,2024-06-30 23:00:00,2024-07-06 05:00:00,"3,493.13","2,003.80",-42.64
21,onyx.box,2024-05-07 20:00:00,2024-05-07 22:00:00,167.68,167.83,0.09
29,watch.box,2024-05-03 13:00:00,2024-05-06 15:00:00,"74,414.36","73,634.98",-1.05
17,good.box,2024-04-28 07:00:00,2024-04-28 07:00:00,314.61,314.61,0.00
18,jane.box,2024-04-26 22:00:00,2024-04-27 00:00:00,"1,155.72","1,151.71",-0.35
0,0000.box,2024-04-23 10:00:00,2024-04-24 04:00:00,91.95,93.92,2.15
19,man.box,2024-04-16 07:00:00,2024-04-28 01:00:00,211.40,224.60,6.25
16,express.box,2024-04-06 22:00:00,2024-04-25 21:00:00,336.25,317.33,-5.63
20,mewtwo.box,2024-03-28 23:00:00,2024-04-17 18:00:00,35.53,29.61,-16.66
2,015.box,2024-02-24 06:00:00,2024-02-25 03:00:00,102.37,104.68,2.26


In [192]:
box_listing_data.columns

Index(['listings', 'max_price', 'min_price', 'avg_price'], dtype='object')

In [193]:
box_listing_data.tail(20)

,listings,max_price,min_price,avg_price
start_time,,,,
2024-06-20,2,"5,399.68",689.40,"3,044.54"
2024-06-21,1,175.09,175.09,175.09
2024-06-22,66,"20,952.36",523.49,"2,267.91"
2024-06-23,0,0.00,0.00,0.00
2024-06-24,23,"246,537.00","22,375.39","42,584.38"
2024-06-25,1,"16,982.50","16,982.50","16,982.50"
2024-06-26,0,0.00,0.00,0.00
2024-06-27,4,"1,692,400.00",611.53,"424,348.94"
2024-06-28,3,"3,430.99",341.54,"1,659.49"


In [194]:
highest_selling_domains

,name,price_usd,price_eth
day,,,
2024-04-27 16:00:00+00:00,vm.box,"31,481.47",10.00
2024-04-27 00:00:00+00:00,jane.box,"1,151.71",0.37
2024-04-25 21:00:00+00:00,express.box,317.33,0.10
2024-04-28 07:00:00+00:00,good.box,314.61,0.10
2024-02-25 07:00:00+00:00,38.box,301.73,0.10
2024-02-25 03:00:00+00:00,70.box,269.17,0.09
2024-02-25 03:00:00+00:00,75.box,269.17,0.09
2024-04-28 01:00:00+00:00,man.box,224.60,0.07
2024-02-25 03:00:00+00:00,40.box,209.36,0.07


In [195]:
highest_selling_domains_fig = px.bar(highest_selling_domains, x=highest_selling_domains['name'], y=highest_selling_domains['price_usd'],
                                     title='10 Highest Selling Domains')
highest_selling_domains_fig.show()

In [196]:


box_listing_data_fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add bar chart for sales
# box_listing_data_fig.add_trace(
#     go.Bar(
#         x=box_listing_data.index,
#         y=box_listing_data['max_price'],
#         name='Max Price'
#     ),
#     secondary_y=False
# )

# box_listing_data_fig.add_trace(
#     go.Bar(
#         x=box_listing_data.index,
#         y=box_listing_data['min_price'],
#         name='Min Price'
#     ),
#     secondary_y=False
# )

# Add line chart for cumulative sales
box_listing_data_fig.add_trace(
    go.Bar(
        x=box_listing_data.index,
        y=box_listing_data['avg_price'],
        name='Avg Price',
    ),
    secondary_y=False
)

box_listing_data_fig.add_trace(
    go.Scatter(
        x=box_listing_data.index,
        y=box_listing_data['listings'],
        name='Listings',
        mode='lines'
    ),
    secondary_y=True
)

box_listing_data_fig.show()


In [197]:
box_listings.drop(columns=['tokenid','eth_usd'], inplace=True)


In [198]:
latest_box_listings = box_listings.sort_index(ascending=False)
latest_box_listings = latest_box_listings.head(10)
latest_box_listings.reset_index(inplace=True)

In [199]:
mint_to_sales_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Mints to Sales Metrics"
)

listing_to_sales_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Listings to Sales Metrics"
)

daily_box_mints_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Daily Mints"
)

daily_box_sales_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Daily Sales"
)

daily_vol_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Daily Volume"
)

daily_sales_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Daily Sales Metrics"
)

daily_mint_metrics_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Daily Mints Metrics"
)

listings_growth_rate_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
    title="Listings Growth Rate"
)

highest_selling_domains_fig.update_layout(
    plot_bgcolor='#fafafa',
    paper_bgcolor='#fafafa',
)



Average Time to Sell a .box Domain: 19 days 11:19:25.506097561

Average Listing Price to Sale Price Change -14.673687659116787

Cumulative Listings to Sales Ratio: 25:1

Cumulative Mints to Sales Ratio: 171:1

Monthly Listings Growth Rate: -70.2247191011236

In [200]:
avg_time_to_sell

Timedelta('19 days 11:19:25.506097561')

In [201]:
key_metrics = [
    {"label": "Average Listing Price to Sale Price Change", "value": f"{round(int(listing_price_to_sale_avg_pct_change),0)}", "unit": "%"},
    {"label": "Average Days on Market", "value": str(avg_time_to_sell.days), "unit": " days"},
    {"label": "Cumulative Listings to Sales Ratio", "value": f"{round(int(cumulative_listings_to_sales), 0)}", "unit": ":1"},
    {"label": "Cumulative Mints to Sales Ratio", "value": f"{round(int(cumulative_mint_to_sales),0)}", "unit": ":1"},
    {"label": "Monthly Listings Growth Rate", "value": f"{listings_growth_rate.iloc[-1]:.2f}", "unit": "%"}
]


In [219]:
max_eth_sale_row

name           vm.box
price_usd   31,481.47
price_eth       10.00
Name: 2024-04-27 16:00:00+00:00, dtype: object

In [226]:
max_eth_sale_details = {
    "name": max_eth_sale_row["name"],
    "price_usd": max_eth_sale_row["price_usd"],
    "price_eth": max_eth_sale_row["price_eth"],
    "date": max_eth_sale_row.name  # This is the index (timestamp)
}

highest_sold_domain_str = f"""
Name: {max_eth_sale_details['name']}
Price (USD): ${max_eth_sale_details['price_usd']:.2f}
Price (ETH): {max_eth_sale_details['price_eth']}
Date: {max_eth_sale_details['date'].strftime('%Y-%m-%d')}
"""

In [222]:
highest_sold_domain_str

'\nName: vm.box\nPrice (USD): $31481.47\nPrice (ETH): 10.0\nDate: 2024-04-27 16:00:00+00:00\n'

In [227]:
sales_metrics = [
    {"label": "Total Sales Volume ", "value":f"${cumulative_box_vol.iloc[-1]:,.2f}", "unit": ""},
    {"label": "Highest Sold Domain", "value": highest_sold_domain_str, "unit": ""},
    {"label": "Average Sales Price ", "value":f"${avg_box_sale:,.2f}", "unit": ""}
    
]

In [203]:
listings_metrics = [

    {"label":"Total .box Listings on Opensea ", "value": str(total_box_listings), "unit": ""}
]

In [204]:
mints_metrics = [
    {"label":"Total .box Mints ", "value":str(total_box_mints), "unit": ""}
]

In [205]:
def generate_table(dataframe, max_rows=11):
    return html.Table([
        html.Thead(
            html.Tr([html.Th(col) for col in dataframe.columns])
        ),
        html.Tbody([
            html.Tr([
                html.Td(dataframe.iloc[i][col]) for col in dataframe.columns
            ]) for i in range(min(len(dataframe), max_rows))
        ])
    ])

In [209]:
latest_box_listings.drop(columns=['Unnamed: 0','chain.1','order_hash.1'], inplace=True)

external_stylesheets = [
    'https://cdnjs.cloudflare.com/ajax/libs/normalize/8.0.1/normalize.min.css', 
    '/assets/styles.css'
]

app = Dash(__name__, external_stylesheets=external_stylesheets)

tts_markdown = f"""
Average Time to Sell a .box Domain: {avg_time_to_sell}
"""

listing_to_sale_mk = f"""Average Listing Price to Sale Price Change {listing_price_to_sale_avg_pct_change} """

cumulative_lts_mk = f"""Cumulative Listings to Sales Ratio: {round(int(cumulative_listings_to_sales), 0)}:1"""
cumulative_mts_mk = f"""Cumulative Mints to Sales Ratio: {round(int(cumulative_mint_to_sales),0)}:1"""
cum_sale_latest_mk = f"""Total Sales Volume: {cumulative_box_vol.iloc[-1]} """
max_sale_mk = f"""Highest Sold Domain: {max_eth_sale_row} """
total_mints_mk = f"""Total .box Mints: {total_box_mints} """
total_listings_mk = f"""Total .box Listings on Opensea {total_box_listings} """
latest_listings_growth_mk = f"""Monthly Listings Growth Rate: {listings_growth_rate.iloc[-1]} """
avg_sold_mk = f"""Average Sales Price: {avg_box_sale} """





# Define the layout of the app
app.layout = html.Div(style={'backgroundColor': 'var(--color-background)'}, children=[
    html.H1(children='.box Domains Dashboard', style={'textAlign': 'center', 'color': 'var(--wcm-color-fg-1)'}),
    html.Br(),
    html.H2('.box Domain Valuator', style={'color': 'var(--wcm-color-fg-2)'}),
    html.Div([
        "Input Domain Name: ",
        dcc.Input(id='valuator-input', value='example.box', type='text'),
        html.Button('Submit', id='submit-button', n_clicks=0)
    ], style={'color': 'var(--wcm-color-fg-1)'}),
    html.Br(),
    html.Div(id='valuator-output', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Br(),
    html.H2('Key Metrics'),
    html.Div(className='metrics-container', style={
        'display': 'grid',
        'gridTemplateColumns': 'repeat(3, 1fr)',  # Adjust based on the number of metrics
        'gap': '20px',
        'padding': '20px',
        'backgroundColor': 'var(--wcm-color-bg-1)',
        'borderRadius': 'var(--wcm-container-border-radius)',
        'boxShadow': '0 4px 8px var(--wcm-color-overlay)'
    }, children=[
        html.Div(className='metric', style={
            'padding': '15px',
            'backgroundColor': 'var(--wcm-color-bg-2)',
            'borderRadius': 'var(--wcm-button-border-radius)',
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)'
        }, children=[
            html.Span(metric["label"], style={'color': 'var(--wcm-color-fg-2)'}),
            html.Span(f"{metric['value']}{metric['unit']}", style={'color': 'var(--wcm-accent-color)'})
        ]) for metric in key_metrics
    ]),
    dcc.Graph(id='mint to sales', figure=mint_to_sales_fig),
    dcc.Graph(id='listings to sales', figure=listing_to_sales_fig),
    
    html.H2('Sales'),
    html.Div(className='metrics-container', style={
        'display': 'grid',
        'gridTemplateColumns': 'repeat(3, 1fr)',  # Adjust based on the number of metrics
        'gap': '20px',
        'padding': '20px',
        'backgroundColor': 'var(--wcm-color-bg-1)',
        'borderRadius': 'var(--wcm-container-border-radius)',
        'boxShadow': '0 4px 8px var(--wcm-color-overlay)'
    }, children=[
        html.Div(className='metric', style={
            'padding': '15px',
            'backgroundColor': 'var(--wcm-color-bg-2)',
            'borderRadius': 'var(--wcm-button-border-radius)',
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)'
        }, children=[
            html.Span(metric["label"], style={'color': 'var(--wcm-color-fg-2)'}),
            html.Span(f"{metric['value']}{metric['unit']}", style={'color': 'var(--wcm-accent-color)'})
        ]) for metric in sales_metrics
    ]),
    dcc.Graph(id='daily_sales_count', figure=daily_sales_fig),
    dcc.Graph(id='daily_sales_vol', figure=daily_vol_fig),
    dcc.Graph(id='highest selling', figure=highest_selling_domains_fig),
    html.Br(),
    html.H3('Monthly Sales Metrics'),
    # generate_table(monthly_box_sales_metrics),
    dash_table.DataTable(
        id='monthly_sales_metrics',
        columns=[{"name": i, "id": i} for i in monthly_box_sales_metrics.columns],
        data=monthly_box_sales_metrics.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),

    html.H3('10 Latest Sales'),
    dash_table.DataTable(
        id='latest_sales',
        columns=[{"name": i, "id": i} for i in latest_box_domains_sales.columns],
        data=latest_box_domains_sales.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),
    html.Br(),
    html.H3('10 Highest Selling Domains'),
    dash_table.DataTable(
        id='highest_sales',
        columns=[{"name": i, "id": i} for i in highest_selling_domains.columns],
        data=highest_selling_domains.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),
    
    html.Br(),
    html.H2('Listings'),
    html.Div(className='metrics-container', style={
        'display': 'grid',
        'gridTemplateColumns': 'repeat(1, 1fr)',  # Adjust based on the number of metrics
        'gap': '20px',
        'padding': '20px',
        'backgroundColor': 'var(--wcm-color-bg-1)',
        'borderRadius': 'var(--wcm-container-border-radius)',
        'boxShadow': '0 4px 8px var(--wcm-color-overlay)'
    }, children=[
        html.Div(className='metric', style={
            'padding': '15px',
            'backgroundColor': 'var(--wcm-color-bg-2)',
            'borderRadius': 'var(--wcm-button-border-radius)',
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)'
        }, children=[
            html.Span(metric["label"], style={'color': 'var(--wcm-color-fg-2)'}),
            html.Span(f"{metric['value']}{metric['unit']}", style={'color': 'var(--wcm-accent-color)'})
        ]) for metric in listings_metrics
    ]),
    dcc.Graph(id='monthly listings growth', figure=listings_growth_rate_fig),
    html.H3('Historical Listings to Sales'),
    dash_table.DataTable(
        id='listings_to_sales',
        columns=[{"name": i, "id": i} for i in historical_listing_to_sales.columns],
        data=historical_listing_to_sales.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),
    html.H3('10 Latest Listings'),
    dash_table.DataTable(
        id='latest_listings',
        columns=[{"name": i, "id": i} for i in latest_box_listings.columns],
        data=latest_box_listings.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),



    html.H2('Mints'),
    html.Div(className='metrics-container', style={
        'display': 'grid',
        'gridTemplateColumns': 'repeat(1, 1fr)',  # Adjust based on the number of metrics
        'gap': '20px',
        'padding': '20px',
        'backgroundColor': 'var(--wcm-color-bg-1)',
        'borderRadius': 'var(--wcm-container-border-radius)',
        'boxShadow': '0 4px 8px var(--wcm-color-overlay)'
    }, children=[
        html.Div(className='metric', style={
            'padding': '15px',
            'backgroundColor': 'var(--wcm-color-bg-2)',
            'borderRadius': 'var(--wcm-button-border-radius)',
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)'
        }, children=[
            html.Span(metric["label"], style={'color': 'var(--wcm-color-fg-2)'}),
            html.Span(f"{metric['value']}{metric['unit']}", style={'color': 'var(--wcm-accent-color)'})
        ]) for metric in mints_metrics
    ]),
    dcc.Graph(id='daily_mints', figure=daily_mint_metrics_fig),
    html.Br(),
    html.H3('10 Latest Mints'),
    dash_table.DataTable(
        id='latest_mints',
        columns=[{"name": i, "id": i} for i in latest_box_domains_mints.columns],
        data=latest_box_domains_mints.to_dict('records'),
        style_table={'overflowX': 'auto'},
        style_cell={
            'height': 'auto',
            'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
            'whiteSpace': 'normal',
            'font-family': 'var(--font-primary, "Inter")',
            'backgroundColor': 'var(--color-background)',
            'color': 'var(--wcm-color-fg-2)'
        }
    ),
    
    
])

# Define the callback
@callback(
    Output(component_id='valuator-output', component_property='children'),
    Input(component_id='submit-button', component_property='n_clicks'),
    State(component_id='valuator-input', component_property='value')
)
def update_output_div(n_clicks, domain):
    if n_clicks == 0:
        return "Please enter a domain name and click Submit."
    
    if not domain:
        return "Please enter a domain name."
    
    domain_df = pd.DataFrame({'domain': [domain]})
    domain_processed = model_prep(domain_df)
    domain_value = value_domain(domain_processed)
    return f'Estimated Value: ${round(domain_value,2):,.2f}'

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [228]:
external_stylesheets = [
    'https://cdnjs.cloudflare.com/ajax/libs/normalize/8.0.1/normalize.min.css', 
    '/assets/styles.css'
]

app = Dash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div(style={'backgroundColor': 'var(--color-background)'}, children=[
    html.H1(
        children='.box Domains Dashboard',
        style={
            'textAlign': 'center',
            'color': 'var(--wcm-color-fg-1)',
            'fontSize': '36px',
            'fontWeight': 'bold',
            'marginBottom': '20px'
        }
    ),
    html.Br(),
    html.H2('.box Domain Valuator', style={
        'color': 'var(--wcm-color-fg-2)', 
        'textAlign': 'center', 
        'marginBottom': '20px'
    }),
    html.Div([
        html.Label("Input Domain Name:", style={
            'color': 'var(--wcm-color-fg-1)', 
            'marginRight': '10px',
            'fontWeight': 'bold'
        }),
        dcc.Input(
            id='valuator-input',
            value='example',
            type='text',
            style={
                'padding': '10px',
                'borderRadius': 'var(--wcm-input-border-radius)',
                'border': '1px solid var(--color-border)',
                'marginRight': '10px'
            },
            pattern='[^.]*'  # Regex pattern to disallow '.' character
        ),
        html.Button('Submit', id='submit-button', n_clicks=0, style={
            'padding': '10px 20px', 
            'borderRadius': 'var(--wcm-button-border-radius)', 
            'backgroundColor': 'var(--wcm-accent-color)', 
            'color': 'var(--wcm-accent-fill-color)',
            'border': 'none',
            'cursor': 'pointer'
        })
    ], style={
        'display': 'flex', 
        'alignItems': 'center', 
        'justifyContent': 'center', 
        'marginBottom': '20px'
    }),
    html.Br(),
    html.Div(id='valuator-output', style={
        'color': 'var(--wcm-color-fg-1)', 
        'textAlign': 'center', 
        'marginTop': '20px'
    }),
    html.Br(),
    html.H2('Key Metrics', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(className='metrics-container', children=[
        html.Div(className='metric', children=[
            html.Span(metric["label"], className='label'),
            html.Span(f"{metric['value']}{metric['unit']}", className='value')
        ]) for metric in key_metrics
    ]),
    html.Br(),
    dcc.Graph(id='mint to sales', figure=mint_to_sales_fig),
    dcc.Graph(id='listings to sales', figure=listing_to_sales_fig),
    
    html.H2('Sales', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(className='metrics-container', children=[
        html.Div(className='metric', children=[
            html.Span(metric["label"], className='label'),
            html.Span(f"{metric['value']}{metric['unit']}", className='value')
        ]) for metric in sales_metrics
    ]),
    html.Br(),
    dcc.Graph(id='daily_sales_count', figure=daily_sales_fig),
    dcc.Graph(id='daily_sales_vol', figure=daily_vol_fig),
    dcc.Graph(id='highest selling', figure=highest_selling_domains_fig),
    html.Br(),
    html.H3('Monthly Sales Metrics', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='monthly_sales_metrics',
                columns=[{"name": i, "id": i} for i in monthly_box_sales_metrics.columns],
                data=monthly_box_sales_metrics.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),
    html.Br(),
    html.H3('10 Latest Sales', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='latest_sales',
                columns=[{"name": i, "id": i} for i in latest_box_domains_sales.columns],
                data=latest_box_domains_sales.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),
    html.Br(),
    html.H3('10 Highest Selling Domains', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='highest_sales',
                columns=[{"name": i, "id": i} for i in highest_selling_domains.columns],
                data=highest_selling_domains.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),
    html.Br(),
    html.H2('Listings', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(className='metrics-container', children=[
        html.Div(className='metric', children=[
            html.Span(metric["label"], className='label'),
            html.Span(f"{metric['value']}{metric['unit']}", className='value')
        ]) for metric in listings_metrics
    ]),
    html.Br(),
    dcc.Graph(id='monthly listings growth', figure=listings_growth_rate_fig),
    html.H3('Historical Listings to Sales', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='listings_to_sales',
                columns=[{"name": i, "id": i} for i in historical_listing_to_sales.columns],
                data=historical_listing_to_sales.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),
    html.H3('10 Latest Listings', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='latest_listings',
                columns=[{"name": i, "id": i} for i in latest_box_listings.columns],
                data=latest_box_listings.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),

    html.H2('Mints', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(className='metrics-container', children=[
        html.Div(className='metric', children=[
            html.Span(metric["label"], className='label'),
            html.Span(f"{metric['value']}{metric['unit']}", className='value')
        ]) for metric in mints_metrics
    ]),
    html.Br(),
    dcc.Graph(id='daily_mints', figure=daily_mint_metrics_fig),
    html.Br(),
    html.H3('10 Latest Mints', style={'color': 'var(--wcm-color-fg-1)'}),
    html.Div(style={'display': 'flex', 'justify-content': 'center', 'padding': '10px'}, children=[
        html.Div(style={'width': '80%', 'max-width': '1000px'}, children=[
            dash_table.DataTable(
                id='latest_mints',
                columns=[{"name": i, "id": i} for i in latest_box_domains_mints.columns],
                data=latest_box_domains_mints.to_dict('records'),
                style_table={'overflowX': 'auto'},
                style_as_list_view=True,
                style_header={
                    'backgroundColor': 'var(--wcm-color-bg-2)',
                    'fontWeight': 'bold',
                    'color': 'var(--wcm-color-fg-1)'
                },
                style_cell={
                    'height': 'auto',
                    'minWidth': '150px', 'width': '150px', 'maxWidth': '150px',
                    'whiteSpace': 'normal',
                    'font-family': 'var(--font-primary, "Inter")',
                    'backgroundColor': 'var(--color-background)',
                    'color': 'var(--wcm-color-fg-2)',
                    'padding': '10px',
                    'border': '1px solid var(--color-border)'
                },
                style_data={
                    'border': '1px solid var(--color-border)',
                    'padding': '10px',
                }
            )
        ])
    ]),
])

# Define the callback
@callback(
    Output(component_id='valuator-output', component_property='children'),
    Input(component_id='submit-button', component_property='n_clicks'),
    State(component_id='valuator-input', component_property='value')
)
def update_output_div(n_clicks, domain_prefix):
    if n_clicks == 0:
        return "Please enter a domain prefix and click Submit."
    
    if not domain_prefix:
        return "Please enter a domain prefix."
    
    if '.' in domain_prefix:
        return "Invalid input. Please enter a valid domain prefix without a '.' character."
    
    domain = f"{domain_prefix}.box"
    domain_df = pd.DataFrame({'domain': [domain]})
    domain_processed = model_prep(domain_df)
    domain_value = value_domain(domain_processed)
    return html.Div([
        html.Div(f'Domain: {domain}', style={'font-weight': 'bold'}),
        html.Div(f'Estimated Value: ${round(domain_value, 2):,.2f}')
    ])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
